# 11. ANOVA (Analysis of Variance)

## Tujuan Pembelajaran
- Memahami konsep ANOVA dan kapan menggunakannya
- Membedakan ANOVA satu arah dan dua arah
- Menghitung F-statistic dan p-value
- Menginterpretasikan hasil ANOVA

## Materi
1. Pengertian ANOVA (Analysis of Variance)
2. Asumsi ANOVA (ANOVA Assumptions)
3. One-way ANOVA (ANOVA Satu Arah)
4. Two-way ANOVA (ANOVA Dua Arah)
5. F-test dan F-distribution
6. Post-hoc Analysis (Tukey HSD, Bonferroni)
7. Aplikasi Praktis ANOVA
8. Best Practices dan Troubleshooting


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from scipy.stats import f_oneway, f
import seaborn as sns
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')

# Set up plotting
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("Libraries imported successfully!")
print("SciPy version:", stats.__version__)


In [ ]:
# Simulasi ANOVA
np.random.seed(42)

# Data untuk One-way ANOVA
group1 = np.random.normal(50, 10, 30)  # Kelompok A
group2 = np.random.normal(55, 10, 30)  # Kelompok B  
group3 = np.random.normal(60, 10, 30)  # Kelompok C
group4 = np.random.normal(52, 10, 30)  # Kelompok D

print("=== ONE-WAY ANOVA ===")
print(f"Kelompok 1: mean={np.mean(group1):.2f}, std={np.std(group1, ddof=1):.2f}, n={len(group1)}")
print(f"Kelompok 2: mean={np.mean(group2):.2f}, std={np.std(group2, ddof=1):.2f}, n={len(group2)}")
print(f"Kelompok 3: mean={np.mean(group3):.2f}, std={np.std(group3, ddof=1):.2f}, n={len(group3)}")
print(f"Kelompok 4: mean={np.mean(group4):.2f}, std={np.std(group4, ddof=1):.2f}, n={len(group4)}")

# One-way ANOVA
f_stat, p_value = f_oneway(group1, group2, group3, group4)

print(f"\nHasil One-way ANOVA:")
print(f"F-statistic: {f_stat:.4f}")
print(f"p-value: {p_value:.4f}")

# Interpretasi
alpha = 0.05
if p_value < alpha:
    print(f"Keputusan: Tolak H0 (p < {alpha})")
    print("Kesimpulan: Minimal ada satu kelompok yang berbeda signifikan")
else:
    print(f"Keputusan: Gagal tolak H0 (p >= {alpha})")
    print("Kesimpulan: Tidak ada perbedaan signifikan antar kelompok")

# Two-way ANOVA (simulasi sederhana)
print("\n=== TWO-WAY ANOVA ===")
# Membuat data untuk two-way ANOVA
n_per_group = 20
factor_a = np.repeat(['A1', 'A2'], n_per_group * 2)
factor_b = np.tile(['B1', 'B2'], n_per_group * 2)

# Data dengan efek interaksi
np.random.seed(42)
values = []
for i, (a, b) in enumerate(zip(factor_a, factor_b)):
    if a == 'A1' and b == 'B1':
        val = np.random.normal(50, 5)
    elif a == 'A1' and b == 'B2':
        val = np.random.normal(55, 5)
    elif a == 'A2' and b == 'B1':
        val = np.random.normal(60, 5)
    else:  # A2, B2
        val = np.random.normal(65, 5)
    values.append(val)

# Membuat DataFrame untuk two-way ANOVA
df_two_way = pd.DataFrame({
    'Factor_A': factor_a,
    'Factor_B': factor_b,
    'Value': values
})

print("Data Two-way ANOVA:")
print(df_two_way.head(10))

# Two-way ANOVA menggunakan statsmodels
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

model = ols('Value ~ C(Factor_A) + C(Factor_B) + C(Factor_A):C(Factor_B)', data=df_two_way).fit()
anova_table = anova_lm(model, typ=2)

print("\nHasil Two-way ANOVA:")
print(anova_table)

# Visualisasi
plt.figure(figsize=(15, 12))

# Plot 1: One-way ANOVA - Box plot
plt.subplot(3, 3, 1)
data_for_box = [group1, group2, group3, group4]
plt.boxplot(data_for_box, labels=['Group 1', 'Group 2', 'Group 3', 'Group 4'])
plt.ylabel('Values')
plt.title('One-way ANOVA - Box Plot')
plt.grid(True, alpha=0.3)

# Plot 2: One-way ANOVA - Histogram
plt.subplot(3, 3, 2)
plt.hist(group1, alpha=0.5, label='Group 1', bins=10)
plt.hist(group2, alpha=0.5, label='Group 2', bins=10)
plt.hist(group3, alpha=0.5, label='Group 3', bins=10)
plt.hist(group4, alpha=0.5, label='Group 4', bins=10)
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('One-way ANOVA - Histogram')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: F-distribution
plt.subplot(3, 3, 3)
df_between = 3  # k-1 = 4-1
df_within = len(group1) + len(group2) + len(group3) + len(group4) - 4  # N-k
x = np.linspace(0, 10, 100)
f_dist = f.pdf(x, df_between, df_within)
plt.plot(x, f_dist, 'b-', linewidth=2, label=f'F-distribution (df1={df_between}, df2={df_within})')
plt.axvline(f_stat, color='red', linestyle='--', linewidth=2, label=f'F-statistic: {f_stat:.2f}')
plt.xlabel('F-value')
plt.ylabel('Density')
plt.title('F-distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Two-way ANOVA - Interaction plot
plt.subplot(3, 3, 4)
for b in ['B1', 'B2']:
    subset = df_two_way[df_two_way['Factor_B'] == b]
    means = subset.groupby('Factor_A')['Value'].mean()
    plt.plot(['A1', 'A2'], means, 'o-', label=f'Factor B: {b}')
plt.xlabel('Factor A')
plt.ylabel('Mean Value')
plt.title('Two-way ANOVA - Interaction Plot')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 5: Two-way ANOVA - Box plot by factors
plt.subplot(3, 3, 5)
df_two_way.boxplot(column='Value', by=['Factor_A', 'Factor_B'], ax=plt.gca())
plt.title('Two-way ANOVA - Box Plot by Factors')
plt.suptitle('')  # Remove default title
plt.grid(True, alpha=0.3)

# Plot 6: Residuals plot
plt.subplot(3, 3, 6)
residuals = model.resid
fitted = model.fittedvalues
plt.scatter(fitted, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.grid(True, alpha=0.3)

# Plot 7: Q-Q plot for residuals
plt.subplot(3, 3, 7)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.grid(True, alpha=0.3)

# Plot 8: Mean comparison
plt.subplot(3, 3, 8)
means = [np.mean(group1), np.mean(group2), np.mean(group3), np.mean(group4)]
groups = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
plt.bar(groups, means, alpha=0.7, color=['red', 'blue', 'green', 'orange'])
plt.ylabel('Mean Value')
plt.title('Group Means Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 9: Effect sizes
plt.subplot(3, 3, 9)
# Calculate eta-squared (effect size)
ss_between = sum([len(group) * (np.mean(group) - np.mean(np.concatenate([group1, group2, group3, group4])))**2 
                  for group in [group1, group2, group3, group4]])
ss_total = sum([(val - np.mean(np.concatenate([group1, group2, group3, group4])))**2 
                for group in [group1, group2, group3, group4] for val in group])
eta_squared = ss_between / ss_total

plt.bar(['Eta-squared'], [eta_squared], alpha=0.7, color='purple')
plt.ylabel('Effect Size')
plt.title(f'Effect Size (η² = {eta_squared:.3f})')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nEffect Size (Eta-squared): {eta_squared:.3f}")
print("Interpretasi Effect Size:")
if eta_squared < 0.01:
    print("Effect size: Negligible")
elif eta_squared < 0.06:
    print("Effect size: Small")
elif eta_squared < 0.14:
    print("Effect size: Medium")
else:
    print("Effect size: Large")


## 1. Pengertian ANOVA (Analysis of Variance)

**ANOVA** adalah teknik statistik yang digunakan untuk menguji apakah ada perbedaan yang signifikan antara rata-rata dari tiga atau lebih kelompok.

### Kapan Menggunakan ANOVA:
- Membandingkan rata-rata dari 3 atau lebih kelompok
- Data kuantitatif (interval/ratio)
- Sampel independen
- Distribusi normal dalam setiap kelompok
- Varian yang sama antar kelompok

### Hipotesis ANOVA:
- **H₀**: μ₁ = μ₂ = μ₃ = ... = μₖ (tidak ada perbedaan rata-rata)
- **H₁**: Minimal ada satu rata-rata yang berbeda

### Konsep Dasar:
- **Between-group variance**: Variasi antar kelompok
- **Within-group variance**: Variasi dalam kelompok
- **F-ratio**: Rasio antara between-group dan within-group variance


## 2. Asumsi ANOVA (ANOVA Assumptions)

**Asumsi ANOVA** adalah kondisi yang harus dipenuhi agar uji ANOVA valid dan dapat diandalkan.

### Asumsi Utama:
1. **Independence**: Observasi harus independen satu sama lain
2. **Normality**: Data dalam setiap kelompok harus berdistribusi normal
3. **Homogeneity of Variance**: Varian antar kelompok harus sama (homoscedasticity)
4. **Random Sampling**: Sampel harus diambil secara acak dari populasi

### Pengecekan Asumsi:
- **Normality**: Shapiro-Wilk test, Kolmogorov-Smirnov test, Q-Q plot
- **Homogeneity**: Levene's test, Bartlett's test, F-test
- **Independence**: Desain penelitian yang tepat

### Pelanggaran Asumsi:
- **Non-normality**: Transformasi data, uji non-parametrik (Kruskal-Wallis)
- **Heteroscedasticity**: Welch's ANOVA, uji non-parametrik
- **Non-independence**: Mixed-effects models, repeated measures ANOVA


In [ ]:
# Demonstrasi Pengecekan Asumsi ANOVA
print("=== DEMONSTRASI PENGECEKAN ASUMSI ANOVA ===")

# Data untuk demonstrasi asumsi
np.random.seed(42)
group1_assumption = np.random.normal(50, 10, 30)
group2_assumption = np.random.normal(55, 10, 30)
group3_assumption = np.random.normal(60, 10, 30)

print("Data untuk pengecekan asumsi:")
print(f"Group 1: mean={np.mean(group1_assumption):.2f}, std={np.std(group1_assumption, ddof=1):.2f}")
print(f"Group 2: mean={np.mean(group2_assumption):.2f}, std={np.std(group2_assumption, ddof=1):.2f}")
print(f"Group 3: mean={np.mean(group3_assumption):.2f}, std={np.std(group3_assumption, ddof=1):.2f}")

# 1. Pengecekan Normalitas
print("\n1. PENGECEKAN NORMALITAS:")
from scipy.stats import shapiro, normaltest, jarque_bera

# Shapiro-Wilk test
shapiro1 = shapiro(group1_assumption)
shapiro2 = shapiro(group2_assumption)
shapiro3 = shapiro(group3_assumption)

print("Shapiro-Wilk Test:")
print(f"  Group 1: W={shapiro1.statistic:.4f}, p={shapiro1.pvalue:.4f}")
print(f"  Group 2: W={shapiro2.statistic:.4f}, p={shapiro2.pvalue:.4f}")
print(f"  Group 3: W={shapiro3.statistic:.4f}, p={shapiro3.pvalue:.4f}")

# D'Agostino's normality test
dagostino1 = normaltest(group1_assumption)
dagostino2 = normaltest(group2_assumption)
dagostino3 = normaltest(group3_assumption)

print("\nD'Agostino's Normality Test:")
print(f"  Group 1: statistic={dagostino1.statistic:.4f}, p={dagostino1.pvalue:.4f}")
print(f"  Group 2: statistic={dagostino2.statistic:.4f}, p={dagostino2.pvalue:.4f}")
print(f"  Group 3: statistic={dagostino3.statistic:.4f}, p={dagostino3.pvalue:.4f}")

# Jarque-Bera test
jb1 = jarque_bera(group1_assumption)
jb2 = jarque_bera(group2_assumption)
jb3 = jarque_bera(group3_assumption)

print("\nJarque-Bera Test:")
print(f"  Group 1: statistic={jb1.statistic:.4f}, p={jb1.pvalue:.4f}")
print(f"  Group 2: statistic={jb2.statistic:.4f}, p={jb2.pvalue:.4f}")
print(f"  Group 3: statistic={jb3.statistic:.4f}, p={jb3.pvalue:.4f}")

# 2. Pengecekan Homogenitas Varian
print("\n2. PENGECEKAN HOMOGENITAS VARIAN:")
from scipy.stats import levene, bartlett

# Levene's test
levene_stat, levene_p = levene(group1_assumption, group2_assumption, group3_assumption)
print(f"Levene's Test: statistic={levene_stat:.4f}, p={levene_p:.4f}")

# Bartlett's test
bartlett_stat, bartlett_p = bartlett(group1_assumption, group2_assumption, group3_assumption)
print(f"Bartlett's Test: statistic={bartlett_stat:.4f}, p={bartlett_p:.4f}")

# F-test untuk homogenitas
var1 = np.var(group1_assumption, ddof=1)
var2 = np.var(group2_assumption, ddof=1)
var3 = np.var(group3_assumption, ddof=1)
max_var = max(var1, var2, var3)
min_var = min(var1, var2, var3)
f_homogeneity = max_var / min_var
print(f"F-test (max/min variance): {f_homogeneity:.4f}")

# 3. Interpretasi Asumsi
print("\n3. INTERPRETASI ASUMSI:")
alpha = 0.05

print("Normalitas (p > 0.05 berarti data normal):")
print(f"  Group 1: {'Normal' if shapiro1.pvalue > alpha else 'Non-normal'} (p={shapiro1.pvalue:.4f})")
print(f"  Group 2: {'Normal' if shapiro2.pvalue > alpha else 'Non-normal'} (p={shapiro2.pvalue:.4f})")
print(f"  Group 3: {'Normal' if shapiro3.pvalue > alpha else 'Non-normal'} (p={shapiro3.pvalue:.4f})")

print(f"\nHomogenitas Varian (p > 0.05 berarti varian sama):")
print(f"  Levene: {'Homogen' if levene_p > alpha else 'Heterogen'} (p={levene_p:.4f})")
print(f"  Bartlett: {'Homogen' if bartlett_p > alpha else 'Heterogen'} (p={bartlett_p:.4f})")

# 4. Visualisasi Pengecekan Asumsi
plt.figure(figsize=(18, 12))

# Plot 1: Q-Q plots untuk normalitas
plt.subplot(3, 4, 1)
stats.probplot(group1_assumption, dist="norm", plot=plt)
plt.title('Q-Q Plot - Group 1')
plt.grid(True, alpha=0.3)

plt.subplot(3, 4, 2)
stats.probplot(group2_assumption, dist="norm", plot=plt)
plt.title('Q-Q Plot - Group 2')
plt.grid(True, alpha=0.3)

plt.subplot(3, 4, 3)
stats.probplot(group3_assumption, dist="norm", plot=plt)
plt.title('Q-Q Plot - Group 3')
plt.grid(True, alpha=0.3)

# Plot 4: Histogram untuk normalitas
plt.subplot(3, 4, 4)
plt.hist(group1_assumption, bins=10, alpha=0.7, color='red', label='Group 1', density=True)
plt.hist(group2_assumption, bins=10, alpha=0.7, color='blue', label='Group 2', density=True)
plt.hist(group3_assumption, bins=10, alpha=0.7, color='green', label='Group 3', density=True)
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Histogram - Normality Check')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 5: Box plots untuk homogenitas
plt.subplot(3, 4, 5)
plt.boxplot([group1_assumption, group2_assumption, group3_assumption], 
           labels=['Group 1', 'Group 2', 'Group 3'])
plt.ylabel('Value')
plt.title('Box Plot - Homogeneity Check')
plt.grid(True, alpha=0.3)

# Plot 6: Residuals vs Fitted (untuk linearity)
plt.subplot(3, 4, 6)
# Simulasi residuals
all_data = np.concatenate([group1_assumption, group2_assumption, group3_assumption])
group_means = [np.mean(group1_assumption), np.mean(group2_assumption), np.mean(group3_assumption)]
fitted_values = np.concatenate([
    np.full(len(group1_assumption), group_means[0]),
    np.full(len(group2_assumption), group_means[1]),
    np.full(len(group3_assumption), group_means[2])
])
residuals = all_data - fitted_values

plt.scatter(fitted_values, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.grid(True, alpha=0.3)

# Plot 7: Variance comparison
plt.subplot(3, 4, 7)
variances = [var1, var2, var3]
groups = ['Group 1', 'Group 2', 'Group 3']
plt.bar(groups, variances, color=['red', 'blue', 'green'], alpha=0.7)
plt.ylabel('Variance')
plt.title('Variance Comparison')
plt.grid(True, alpha=0.3)

# Plot 8: Standard deviation comparison
plt.subplot(3, 4, 8)
stds = [np.std(group1_assumption, ddof=1), np.std(group2_assumption, ddof=1), np.std(group3_assumption, ddof=1)]
plt.bar(groups, stds, color=['red', 'blue', 'green'], alpha=0.7)
plt.ylabel('Standard Deviation')
plt.title('Standard Deviation Comparison')
plt.grid(True, alpha=0.3)

# Plot 9: Mean comparison
plt.subplot(3, 4, 9)
means = [np.mean(group1_assumption), np.mean(group2_assumption), np.mean(group3_assumption)]
plt.bar(groups, means, color=['red', 'blue', 'green'], alpha=0.7)
plt.ylabel('Mean')
plt.title('Mean Comparison')
plt.grid(True, alpha=0.3)

# Plot 10: Distribution comparison
plt.subplot(3, 4, 10)
x = np.linspace(min(all_data), max(all_data), 100)
y1 = stats.norm.pdf(x, np.mean(group1_assumption), np.std(group1_assumption, ddof=1))
y2 = stats.norm.pdf(x, np.mean(group2_assumption), np.std(group2_assumption, ddof=1))
y3 = stats.norm.pdf(x, np.mean(group3_assumption), np.std(group3_assumption, ddof=1))

plt.plot(x, y1, 'r-', linewidth=2, label='Group 1')
plt.plot(x, y2, 'b-', linewidth=2, label='Group 2')
plt.plot(x, y3, 'g-', linewidth=2, label='Group 3')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Distribution Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 11: Cook's distance (untuk outlier detection)
plt.subplot(3, 4, 11)
# Simulasi Cook's distance
cooks_d = np.random.exponential(0.1, len(all_data))  # Simulasi
plt.scatter(range(len(cooks_d)), cooks_d, alpha=0.7)
plt.axhline(0.5, color='red', linestyle='--', label='Threshold')
plt.xlabel('Observation')
plt.ylabel("Cook's Distance")
plt.title("Cook's Distance")
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 12: Summary statistics
plt.subplot(3, 4, 12)
plt.axis('off')
summary_text = f"""
ASUMSI ANOVA CHECK

Normalitas (Shapiro-Wilk):
Group 1: p = {shapiro1.pvalue:.4f}
Group 2: p = {shapiro2.pvalue:.4f}
Group 3: p = {shapiro3.pvalue:.4f}

Homogenitas Varian:
Levene: p = {levene_p:.4f}
Bartlett: p = {bartlett_p:.4f}

Kesimpulan:
Normalitas: {'OK' if all(p > alpha for p in [shapiro1.pvalue, shapiro2.pvalue, shapiro3.pvalue]) else 'VIOLATED'}
Homogenitas: {'OK' if levene_p > alpha else 'VIOLATED'}
"""
plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, fontsize=10, 
         verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# 5. Rekomendasi berdasarkan asumsi
print("\n5. REKOMENDASI BERDASARKAN ASUMSI:")
if all(p > alpha for p in [shapiro1.pvalue, shapiro2.pvalue, shapiro3.pvalue]) and levene_p > alpha:
    print("✓ Semua asumsi terpenuhi - ANOVA dapat dilakukan")
elif levene_p <= alpha:
    print("⚠ Homogenitas varian dilanggar - Gunakan Welch's ANOVA atau Kruskal-Wallis")
elif not all(p > alpha for p in [shapiro1.pvalue, shapiro2.pvalue, shapiro3.pvalue]):
    print("⚠ Normalitas dilanggar - Gunakan Kruskal-Wallis atau transformasi data")
else:
    print("✓ Asumsi terpenuhi - ANOVA dapat dilakukan")


## 3. One-way ANOVA (ANOVA Satu Arah)

**One-way ANOVA** adalah uji statistik yang digunakan untuk membandingkan rata-rata dari tiga atau lebih kelompok independen.

### Rumus One-way ANOVA:
```
F = MS_between / MS_within
```

Dimana:
- **MS_between** = SS_between / df_between
- **MS_within** = SS_within / df_within
- **SS_between** = Σ n_i (x̄_i - x̄_total)²
- **SS_within** = Σ Σ (x_ij - x̄_i)²
- **df_between** = k - 1 (k = jumlah kelompok)
- **df_within** = N - k (N = total observasi)

### Langkah-langkah One-way ANOVA:
1. **Hipotesis**:
   - H₀: μ₁ = μ₂ = μ₃ = ... = μₖ
   - H₁: Minimal ada satu μ yang berbeda

2. **Hitung F-statistic**:
   - F = MS_between / MS_within

3. **Keputusan**:
   - Tolak H₀ jika F > F_critical atau p-value < α

4. **Interpretasi**:
   - Jika H₀ ditolak, lakukan post-hoc analysis


In [ ]:
# Demonstrasi One-way ANOVA Lengkap
print("=== DEMONSTRASI ONE-WAY ANOVA LENGKAP ===")

# Data untuk demonstrasi one-way ANOVA
np.random.seed(42)

# Simulasi data dengan 4 kelompok
group1 = np.random.normal(50, 10, 30)  # Kelompok A
group2 = np.random.normal(55, 10, 30)  # Kelompok B  
group3 = np.random.normal(60, 10, 30)  # Kelompok C
group4 = np.random.normal(52, 10, 30)  # Kelompok D

print("Data untuk One-way ANOVA:")
print(f"Kelompok 1: mean={np.mean(group1):.2f}, std={np.std(group1, ddof=1):.2f}, n={len(group1)}")
print(f"Kelompok 2: mean={np.mean(group2):.2f}, std={np.std(group2, ddof=1):.2f}, n={len(group2)}")
print(f"Kelompok 3: mean={np.mean(group3):.2f}, std={np.std(group3, ddof=1):.2f}, n={len(group3)}")
print(f"Kelompok 4: mean={np.mean(group4):.2f}, std={np.std(group4, ddof=1):.2f}, n={len(group4)}")

# 1. Perhitungan Manual One-way ANOVA
print("\n1. PERHITUNGAN MANUAL ONE-WAY ANOVA:")

# Gabungkan semua data
all_data = np.concatenate([group1, group2, group3, group4])
grand_mean = np.mean(all_data)

# Hitung SS_between
group_means = [np.mean(group1), np.mean(group2), np.mean(group3), np.mean(group4)]
group_sizes = [len(group1), len(group2), len(group3), len(group4)]

ss_between = sum([n * (mean - grand_mean)**2 for n, mean in zip(group_sizes, group_means)])
print(f"SS_between: {ss_between:.2f}")

# Hitung SS_within
ss_within = sum([sum((group - np.mean(group))**2) for group in [group1, group2, group3, group4]])
print(f"SS_within: {ss_within:.2f}")

# Hitung SS_total
ss_total = sum((all_data - grand_mean)**2)
print(f"SS_total: {ss_total:.2f}")

# Hitung degrees of freedom
k = 4  # jumlah kelompok
N = len(all_data)  # total observasi
df_between = k - 1
df_within = N - k
df_total = N - 1

print(f"df_between: {df_between}")
print(f"df_within: {df_within}")
print(f"df_total: {df_total}")

# Hitung Mean Squares
ms_between = ss_between / df_between
ms_within = ss_within / df_within

print(f"MS_between: {ms_between:.2f}")
print(f"MS_within: {ms_within:.2f}")

# Hitung F-statistic
f_statistic_manual = ms_between / ms_within
print(f"F-statistic: {f_statistic_manual:.4f}")

# Hitung p-value
p_value_manual = 1 - f.cdf(f_statistic_manual, df_between, df_within)
print(f"p-value: {p_value_manual:.4f}")

# 2. Perhitungan dengan SciPy
print("\n2. PERHITUNGAN DENGAN SCIPY:")
f_stat_scipy, p_value_scipy = f_oneway(group1, group2, group3, group4)
print(f"F-statistic (SciPy): {f_stat_scipy:.4f}")
print(f"p-value (SciPy): {p_value_scipy:.4f}")

# 3. Perhitungan dengan statsmodels
print("\n3. PERHITUNGAN DENGAN STATSMODELS:")
# Buat DataFrame
df_anova = pd.DataFrame({
    'value': all_data,
    'group': ['Group1'] * len(group1) + ['Group2'] * len(group2) + 
             ['Group3'] * len(group3) + ['Group4'] * len(group4)
})

# One-way ANOVA dengan statsmodels
model = ols('value ~ C(group)', data=df_anova).fit()
anova_table = anova_lm(model, typ=2)
print("ANOVA Table (statsmodels):")
print(anova_table)

# 4. Effect Size (Eta-squared)
print("\n4. EFFECT SIZE (ETA-SQUARED):")
eta_squared = ss_between / ss_total
print(f"Eta-squared (η²): {eta_squared:.4f}")

# Interpretasi effect size
if eta_squared < 0.01:
    effect_size = "Negligible"
elif eta_squared < 0.06:
    effect_size = "Small"
elif eta_squared < 0.14:
    effect_size = "Medium"
else:
    effect_size = "Large"

print(f"Effect size: {effect_size}")

# 5. Post-hoc Analysis (Tukey HSD)
print("\n5. POST-HOC ANALYSIS (TUKEY HSD):")
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Tukey HSD test
tukey_result = pairwise_tukeyhsd(endog=all_data, groups=df_anova['group'], alpha=0.05)
print("Tukey HSD Results:")
print(tukey_result)

# 6. Visualisasi One-way ANOVA
plt.figure(figsize=(20, 15))

# Plot 1: Box plot
plt.subplot(3, 4, 1)
data_for_box = [group1, group2, group3, group4]
box_plot = plt.boxplot(data_for_box, labels=['Group 1', 'Group 2', 'Group 3', 'Group 4'], patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
plt.ylabel('Values')
plt.title('One-way ANOVA - Box Plot')
plt.grid(True, alpha=0.3)

# Plot 2: Histogram per kelompok
plt.subplot(3, 4, 2)
plt.hist(group1, alpha=0.5, label='Group 1', bins=10, color='blue')
plt.hist(group2, alpha=0.5, label='Group 2', bins=10, color='green')
plt.hist(group3, alpha=0.5, label='Group 3', bins=10, color='red')
plt.hist(group4, alpha=0.5, label='Group 4', bins=10, color='orange')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('One-way ANOVA - Histogram')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: F-distribution
plt.subplot(3, 4, 3)
x = np.linspace(0, 10, 100)
f_dist = f.pdf(x, df_between, df_within)
plt.plot(x, f_dist, 'b-', linewidth=2, label=f'F-distribution (df1={df_between}, df2={df_within})')
plt.axvline(f_statistic_manual, color='red', linestyle='--', linewidth=2, label=f'F-statistic: {f_statistic_manual:.2f}')
plt.axvline(f.ppf(0.95, df_between, df_within), color='green', linestyle=':', linewidth=2, label=f'F-critical: {f.ppf(0.95, df_between, df_within):.2f}')
plt.xlabel('F-value')
plt.ylabel('Density')
plt.title('F-distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: Mean comparison
plt.subplot(3, 4, 4)
means = [np.mean(group1), np.mean(group2), np.mean(group3), np.mean(group4)]
groups = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
bars = plt.bar(groups, means, alpha=0.7, color=['blue', 'green', 'red', 'orange'])
plt.ylabel('Mean Value')
plt.title('Group Means Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Add mean values on bars
for bar, mean in zip(bars, means):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{mean:.2f}', ha='center', va='bottom')

# Plot 5: Residuals plot
plt.subplot(3, 4, 5)
# Simulasi residuals
group_means_array = np.concatenate([
    np.full(len(group1), group_means[0]),
    np.full(len(group2), group_means[1]),
    np.full(len(group3), group_means[2]),
    np.full(len(group4), group_means[3])
])
residuals = all_data - group_means_array
plt.scatter(group_means_array, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.grid(True, alpha=0.3)

# Plot 6: Q-Q plot for residuals
plt.subplot(3, 4, 6)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.grid(True, alpha=0.3)

# Plot 7: Variance comparison
plt.subplot(3, 4, 7)
variances = [np.var(group1, ddof=1), np.var(group2, ddof=1), np.var(group3, ddof=1), np.var(group4, ddof=1)]
plt.bar(groups, variances, alpha=0.7, color=['blue', 'green', 'red', 'orange'])
plt.ylabel('Variance')
plt.title('Variance Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 8: Standard deviation comparison
plt.subplot(3, 4, 8)
stds = [np.std(group1, ddof=1), np.std(group2, ddof=1), np.std(group3, ddof=1), np.std(group4, ddof=1)]
plt.bar(groups, stds, alpha=0.7, color=['blue', 'green', 'red', 'orange'])
plt.ylabel('Standard Deviation')
plt.title('Standard Deviation Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 9: Distribution comparison
plt.subplot(3, 4, 9)
x = np.linspace(min(all_data), max(all_data), 100)
y1 = stats.norm.pdf(x, np.mean(group1), np.std(group1, ddof=1))
y2 = stats.norm.pdf(x, np.mean(group2), np.std(group2, ddof=1))
y3 = stats.norm.pdf(x, np.mean(group3), np.std(group3, ddof=1))
y4 = stats.norm.pdf(x, np.mean(group4), np.std(group4, ddof=1))

plt.plot(x, y1, 'b-', linewidth=2, label='Group 1')
plt.plot(x, y2, 'g-', linewidth=2, label='Group 2')
plt.plot(x, y3, 'r-', linewidth=2, label='Group 3')
plt.plot(x, y4, 'orange', linewidth=2, label='Group 4')
plt.xlabel('Value')
plt.ylabel('Density')
plt.title('Distribution Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 10: ANOVA Summary
plt.subplot(3, 4, 10)
plt.axis('off')
summary_text = f"""
ONE-WAY ANOVA SUMMARY

F-statistic: {f_statistic_manual:.4f}
p-value: {p_value_scipy:.4f}
Eta-squared: {eta_squared:.4f}

Degrees of Freedom:
Between: {df_between}
Within: {df_within}

Mean Squares:
Between: {ms_between:.2f}
Within: {ms_within:.2f}

Effect Size: {effect_size}
"""
plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, fontsize=10, 
         verticalalignment='top', fontfamily='monospace')

# Plot 11: Tukey HSD visualization
plt.subplot(3, 4, 11)
# Simulasi hasil Tukey HSD
tukey_means = means
tukey_errors = [np.std(group, ddof=1)/np.sqrt(len(group)) for group in [group1, group2, group3, group4]]
plt.errorbar(groups, tukey_means, yerr=tukey_errors, fmt='o', capsize=5, capthick=2, 
             color='blue', markersize=8)
plt.ylabel('Mean ± SE')
plt.title('Tukey HSD - Group Means')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 12: Power Analysis
plt.subplot(3, 4, 12)
# Simulasi power analysis
effect_sizes = np.linspace(0.1, 1.0, 100)
powers = []
for es in effect_sizes:
    # Simulasi power calculation
    power = 1 - f.cdf(f.ppf(0.95, df_between, df_within), df_between, df_within)
    powers.append(power)

plt.plot(effect_sizes, powers, 'b-', linewidth=2)
plt.axhline(0.8, color='red', linestyle='--', label='Power = 0.8')
plt.xlabel('Effect Size')
plt.ylabel('Power')
plt.title('Power Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 7. Interpretasi Hasil
print("\n7. INTERPRETASI HASIL:")
alpha = 0.05
if p_value_scipy < alpha:
    print(f"Keputusan: Tolak H0 (p = {p_value_scipy:.4f} < {alpha})")
    print("Kesimpulan: Minimal ada satu kelompok yang berbeda signifikan")
    print("Rekomendasi: Lakukan post-hoc analysis untuk mengetahui kelompok mana yang berbeda")
else:
    print(f"Keputusan: Gagal tolak H0 (p = {p_value_scipy:.4f} >= {alpha})")
    print("Kesimpulan: Tidak ada perbedaan signifikan antar kelompok")

print(f"\nEffect Size: {effect_size} (η² = {eta_squared:.4f})")
print(f"Power: {powers[0]:.3f} (estimated)")

# 8. Kesimpulan dan Rekomendasi
print("\n8. KESIMPULAN DAN REKOMENDASI:")
print("   - One-way ANOVA: Uji perbedaan rata-rata 3+ kelompok independen")
print("   - Asumsi: Normalitas, homogenitas varian, independensi")
print("   - Interpretasi: F-statistic dan p-value untuk keputusan")
print("   - Effect size: Eta-squared untuk kekuatan hubungan")
print("   - Post-hoc: Tukey HSD untuk perbandingan pairwise")
print("   - Visualisasi: Box plot, histogram, residual plots")
print("   - Power analysis: Penting untuk perencanaan penelitian")


## 4. Two-way ANOVA (ANOVA Dua Arah)

### 4.1 Pengertian Two-way ANOVA

**Two-way ANOVA** adalah uji statistik untuk menganalisis perbedaan rata-rata dari dua atau lebih kelompok yang dipengaruhi oleh dua faktor independen. Uji ini memungkinkan kita untuk:

- Menguji efek utama (main effects) dari setiap faktor
- Menguji efek interaksi (interaction effects) antara kedua faktor
- Menguji apakah ada perbedaan signifikan antar kelompok

### 4.2 Komponen Two-way ANOVA

#### 4.2.1 Faktor (Factors)
- **Faktor A**: Variabel independen pertama (misal: metode pembelajaran)
- **Faktor B**: Variabel independen kedua (misal: tingkat kemampuan)
- **Level**: Nilai atau kategori dalam setiap faktor

#### 4.2.2 Efek (Effects)
- **Efek Utama A**: Pengaruh faktor A terhadap variabel dependen
- **Efek Utama B**: Pengaruh faktor B terhadap variabel dependen
- **Efek Interaksi A×B**: Pengaruh interaksi antara faktor A dan B

### 4.3 Model Two-way ANOVA

#### 4.3.1 Model Matematis
```
Yijk = μ + αi + βj + (αβ)ij + εijk
```

Dimana:
- `Yijk` = nilai observasi ke-k pada level i faktor A dan level j faktor B
- `μ` = grand mean (rata-rata keseluruhan)
- `αi` = efek level i faktor A
- `βj` = efek level j faktor B
- `(αβ)ij` = efek interaksi antara level i faktor A dan level j faktor B
- `εijk` = error term

#### 4.3.2 Hipotesis
- **H0A**: Tidak ada efek utama faktor A (α1 = α2 = ... = αa = 0)
- **H1A**: Ada efek utama faktor A (minimal satu αi ≠ 0)
- **H0B**: Tidak ada efek utama faktor B (β1 = β2 = ... = βb = 0)
- **H1B**: Ada efek utama faktor B (minimal satu βj ≠ 0)
- **H0AB**: Tidak ada efek interaksi A×B ((αβ)ij = 0 untuk semua i,j)
- **H1AB**: Ada efek interaksi A×B (minimal satu (αβ)ij ≠ 0)

### 4.4 Tabel ANOVA Two-way

| Sumber Variasi | SS | df | MS | F | p-value |
|----------------|----|----|----|----|---------|
| Faktor A | SSA | a-1 | MSA | MSA/MSE | pA |
| Faktor B | SSB | b-1 | MSB | MSB/MSE | pB |
| Interaksi A×B | SSAB | (a-1)(b-1) | MSAB | MSAB/MSE | pAB |
| Error | SSE | N-ab | MSE | - | - |
| Total | SST | N-1 | - | - | - |

### 4.5 Asumsi Two-way ANOVA

1. **Normalitas**: Data berdistribusi normal dalam setiap sel
2. **Homogenitas Varian**: Varian sama di semua sel
3. **Independensi**: Observasi independen
4. **Additivitas**: Efek faktor bersifat aditif (tidak ada interaksi)

### 4.6 Interpretasi Hasil

#### 4.6.1 Efek Utama
- **Signifikan**: Faktor tersebut mempengaruhi variabel dependen
- **Tidak Signifikan**: Faktor tersebut tidak mempengaruhi variabel dependen

#### 4.6.2 Efek Interaksi
- **Signifikan**: Efek satu faktor bergantung pada level faktor lain
- **Tidak Signifikan**: Efek faktor independen satu sama lain

#### 4.6.3 Effect Size
- **Eta-squared (η²)**: Proporsi varians yang dijelaskan oleh faktor
- **Partial Eta-squared**: Proporsi varians yang dijelaskan oleh faktor setelah mengontrol faktor lain

### 4.7 Aplikasi Two-way ANOVA

1. **Penelitian Pendidikan**: Metode pembelajaran × tingkat kemampuan
2. **Penelitian Medis**: Pengobatan × jenis kelamin
3. **Penelitian Psikologi**: Terapi × usia
4. **Penelitian Bisnis**: Strategi pemasaran × segmentasi pasar
5. **Penelitian Pertanian**: Pupuk × jenis tanah

### 4.8 Keuntungan Two-way ANOVA

1. **Efisiensi**: Menguji dua faktor sekaligus
2. **Interaksi**: Mendeteksi efek interaksi
3. **Kontrol**: Mengontrol efek satu faktor saat menguji faktor lain
4. **Power**: Lebih powerful daripada multiple one-way ANOVA
5. **Interpretasi**: Memberikan pemahaman yang lebih komprehensif

### 4.9 Keterbatasan Two-way ANOVA

1. **Kompleksitas**: Lebih kompleks daripada one-way ANOVA
2. **Asumsi**: Lebih banyak asumsi yang harus dipenuhi
3. **Sample Size**: Membutuhkan sample size yang lebih besar
4. **Interpretasi**: Interpretasi interaksi bisa sulit
5. **Post-hoc**: Analisis post-hoc lebih kompleks

### 4.10 Best Practices

1. **Perencanaan**: Rencanakan desain eksperimen dengan baik
2. **Sample Size**: Hitung sample size yang memadai
3. **Asumsi**: Periksa asumsi sebelum analisis
4. **Visualisasi**: Gunakan grafik untuk memahami data
5. **Interpretasi**: Interpretasikan hasil dengan hati-hati
6. **Reporting**: Laporkan semua hasil termasuk effect size
7. **Post-hoc**: Lakukan post-hoc analysis jika diperlukan


In [ ]:
# Demonstrasi Two-way ANOVA Lengkap
print("=== DEMONSTRASI TWO-WAY ANOVA LENGKAP ===")

# Data untuk demonstrasi two-way ANOVA
np.random.seed(42)

# Simulasi data dengan 2 faktor
# Faktor A: Metode Pembelajaran (3 level)
# Faktor B: Tingkat Kemampuan (2 level)
# Total: 3 × 2 = 6 kelompok

# Data simulasi
data = {
    'score': [],
    'method': [],
    'ability': []
}

# Metode Pembelajaran A1, A2, A3
# Tingkat Kemampuan B1, B2
methods = ['A1', 'A2', 'A3']
abilities = ['B1', 'B2']

# Simulasi data untuk setiap kombinasi
for method in methods:
    for ability in abilities:
        if method == 'A1' and ability == 'B1':
            scores = np.random.normal(75, 8, 20)
        elif method == 'A1' and ability == 'B2':
            scores = np.random.normal(85, 8, 20)
        elif method == 'A2' and ability == 'B1':
            scores = np.random.normal(80, 8, 20)
        elif method == 'A2' and ability == 'B2':
            scores = np.random.normal(90, 8, 20)
        elif method == 'A3' and ability == 'B1':
            scores = np.random.normal(70, 8, 20)
        else:  # A3, B2
            scores = np.random.normal(95, 8, 20)
        
        data['score'].extend(scores)
        data['method'].extend([method] * len(scores))
        data['ability'].extend([ability] * len(scores))

# Buat DataFrame
df_two_way = pd.DataFrame(data)

print("Data untuk Two-way ANOVA:")
print(df_two_way.head(10))
print(f"\nTotal observasi: {len(df_two_way)}")
print(f"Kombinasi faktor: {df_two_way.groupby(['method', 'ability']).size()}")

# 1. Perhitungan Manual Two-way ANOVA
print("\n1. PERHITUNGAN MANUAL TWO-WAY ANOVA:")

# Hitung grand mean
grand_mean = df_two_way['score'].mean()
print(f"Grand mean: {grand_mean:.2f}")

# Hitung mean untuk setiap sel
cell_means = df_two_way.groupby(['method', 'ability'])['score'].mean().unstack()
print("\nCell means:")
print(cell_means)

# Hitung mean untuk setiap level faktor A (method)
method_means = df_two_way.groupby('method')['score'].mean()
print(f"\nMethod means: {method_means.to_dict()}")

# Hitung mean untuk setiap level faktor B (ability)
ability_means = df_two_way.groupby('ability')['score'].mean()
print(f"Ability means: {ability_means.to_dict()}")

# Hitung SS_total
ss_total = sum((df_two_way['score'] - grand_mean)**2)
print(f"SS_total: {ss_total:.2f}")

# Hitung SS_method (Faktor A)
ss_method = sum([len(df_two_way[df_two_way['method'] == method]) * (mean - grand_mean)**2 
                for method, mean in method_means.items()])
print(f"SS_method: {ss_method:.2f}")

# Hitung SS_ability (Faktor B)
ss_ability = sum([len(df_two_way[df_two_way['ability'] == ability]) * (mean - grand_mean)**2 
                 for ability, mean in ability_means.items()])
print(f"SS_ability: {ss_ability:.2f}")

# Hitung SS_cells (sel)
ss_cells = sum([len(df_two_way[(df_two_way['method'] == method) & 
                               (df_two_way['ability'] == ability)]) * (mean - grand_mean)**2
               for method in methods for ability in abilities
               for mean in [df_two_way[(df_two_way['method'] == method) & 
                                      (df_two_way['ability'] == ability)]['score'].mean()]])

# Hitung SS_interaction
ss_interaction = ss_cells - ss_method - ss_ability
print(f"SS_interaction: {ss_interaction:.2f}")

# Hitung SS_error
ss_error = ss_total - ss_cells
print(f"SS_error: {ss_error:.2f}")

# Hitung degrees of freedom
a = len(methods)  # jumlah level faktor A
b = len(abilities)  # jumlah level faktor B
n = len(df_two_way) // (a * b)  # jumlah observasi per sel
N = len(df_two_way)  # total observasi

df_method = a - 1
df_ability = b - 1
df_interaction = (a - 1) * (b - 1)
df_error = N - (a * b)
df_total = N - 1

print(f"\nDegrees of freedom:")
print(f"df_method: {df_method}")
print(f"df_ability: {df_ability}")
print(f"df_interaction: {df_interaction}")
print(f"df_error: {df_error}")
print(f"df_total: {df_total}")

# Hitung Mean Squares
ms_method = ss_method / df_method
ms_ability = ss_ability / df_ability
ms_interaction = ss_interaction / df_interaction
ms_error = ss_error / df_error

print(f"\nMean Squares:")
print(f"MS_method: {ms_method:.2f}")
print(f"MS_ability: {ms_ability:.2f}")
print(f"MS_interaction: {ms_interaction:.2f}")
print(f"MS_error: {ms_error:.2f}")

# Hitung F-statistics
f_method = ms_method / ms_error
f_ability = ms_ability / ms_error
f_interaction = ms_interaction / ms_error

print(f"\nF-statistics:")
print(f"F_method: {f_method:.4f}")
print(f"F_ability: {f_ability:.4f}")
print(f"F_interaction: {f_interaction:.4f}")

# Hitung p-values
p_method = 1 - f.cdf(f_method, df_method, df_error)
p_ability = 1 - f.cdf(f_ability, df_ability, df_error)
p_interaction = 1 - f.cdf(f_interaction, df_interaction, df_error)

print(f"\nP-values:")
print(f"p_method: {p_method:.4f}")
print(f"p_ability: {p_ability:.4f}")
print(f"p_interaction: {p_interaction:.4f}")

# 2. Perhitungan dengan statsmodels
print("\n2. PERHITUNGAN DENGAN STATSMODELS:")

# Two-way ANOVA dengan statsmodels
model_two_way = ols('score ~ C(method) + C(ability) + C(method):C(ability)', data=df_two_way).fit()
anova_table_two_way = anova_lm(model_two_way, typ=2)
print("Two-way ANOVA Table (statsmodels):")
print(anova_table_two_way)

# 3. Effect Size
print("\n3. EFFECT SIZE:")

# Eta-squared
eta_squared_method = ss_method / ss_total
eta_squared_ability = ss_ability / ss_total
eta_squared_interaction = ss_interaction / ss_total

print(f"Eta-squared method: {eta_squared_method:.4f}")
print(f"Eta-squared ability: {eta_squared_ability:.4f}")
print(f"Eta-squared interaction: {eta_squared_interaction:.4f}")

# Partial eta-squared
partial_eta_squared_method = ss_method / (ss_method + ss_error)
partial_eta_squared_ability = ss_ability / (ss_ability + ss_error)
partial_eta_squared_interaction = ss_interaction / (ss_interaction + ss_error)

print(f"\nPartial Eta-squared method: {partial_eta_squared_method:.4f}")
print(f"Partial Eta-squared ability: {partial_eta_squared_ability:.4f}")
print(f"Partial Eta-squared interaction: {partial_eta_squared_interaction:.4f}")

# 4. Visualisasi Two-way ANOVA
plt.figure(figsize=(20, 15))

# Plot 1: Interaction plot
plt.subplot(3, 4, 1)
for ability in abilities:
    ability_data = df_two_way[df_two_way['ability'] == ability]
    method_means_ability = ability_data.groupby('method')['score'].mean()
    plt.plot(methods, method_means_ability, marker='o', linewidth=2, label=f'Ability {ability}')

plt.xlabel('Method')
plt.ylabel('Mean Score')
plt.title('Interaction Plot - Method × Ability')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Box plot per kombinasi
plt.subplot(3, 4, 2)
df_two_way['combination'] = df_two_way['method'] + ' × ' + df_two_way['ability']
df_two_way.boxplot(column='score', by='combination', ax=plt.gca())
plt.title('Box Plot per Kombinasi')
plt.xticks(rotation=45)
plt.suptitle('')  # Remove default title
plt.grid(True, alpha=0.3)

# Plot 3: Heatmap cell means
plt.subplot(3, 4, 3)
sns.heatmap(cell_means, annot=True, fmt='.2f', cmap='YlOrRd', ax=plt.gca())
plt.title('Heatmap Cell Means')
plt.xlabel('Ability')
plt.ylabel('Method')

# Plot 4: Main effects plot - Method
plt.subplot(3, 4, 4)
method_means.plot(kind='bar', ax=plt.gca(), color=['skyblue', 'lightgreen', 'lightcoral'])
plt.title('Main Effect - Method')
plt.xlabel('Method')
plt.ylabel('Mean Score')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)

# Plot 5: Main effects plot - Ability
plt.subplot(3, 4, 5)
ability_means.plot(kind='bar', ax=plt.gca(), color=['orange', 'purple'])
plt.title('Main Effect - Ability')
plt.xlabel('Ability')
plt.ylabel('Mean Score')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3)

# Plot 6: Residuals plot
plt.subplot(3, 4, 6)
fitted_values = model_two_way.fittedvalues
residuals = model_two_way.resid
plt.scatter(fitted_values, residuals, alpha=0.7)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Fitted Values')
plt.ylabel('Residuals')
plt.title('Residuals vs Fitted')
plt.grid(True, alpha=0.3)

# Plot 7: Q-Q plot for residuals
plt.subplot(3, 4, 7)
stats.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.grid(True, alpha=0.3)

# Plot 8: Variance comparison per sel
plt.subplot(3, 4, 8)
cell_vars = df_two_way.groupby(['method', 'ability'])['score'].var().unstack()
sns.heatmap(cell_vars, annot=True, fmt='.2f', cmap='Blues', ax=plt.gca())
plt.title('Variance per Cell')
plt.xlabel('Ability')
plt.ylabel('Method')

# Plot 9: Distribution per sel
plt.subplot(3, 4, 9)
for i, method in enumerate(methods):
    for j, ability in enumerate(abilities):
        cell_data = df_two_way[(df_two_way['method'] == method) & 
                              (df_two_way['ability'] == ability)]['score']
        plt.hist(cell_data, alpha=0.5, bins=10, 
                label=f'{method}×{ability}', 
                color=plt.cm.Set3(i*2 + j))

plt.xlabel('Score')
plt.ylabel('Frequency')
plt.title('Distribution per Cell')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, alpha=0.3)

# Plot 10: ANOVA Summary
plt.subplot(3, 4, 10)
plt.axis('off')
summary_text = f"""
TWO-WAY ANOVA SUMMARY

F-statistics:
Method: {f_method:.4f} (p={p_method:.4f})
Ability: {f_ability:.4f} (p={p_ability:.4f})
Interaction: {f_interaction:.4f} (p={p_interaction:.4f})

Effect Sizes (η²):
Method: {eta_squared_method:.4f}
Ability: {eta_squared_ability:.4f}
Interaction: {eta_squared_interaction:.4f}

Degrees of Freedom:
Method: {df_method}
Ability: {df_ability}
Interaction: {df_interaction}
Error: {df_error}
"""
plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, fontsize=9, 
         verticalalignment='top', fontfamily='monospace')

# Plot 11: Post-hoc comparison
plt.subplot(3, 4, 11)
# Simulasi post-hoc results
from itertools import combinations
method_pairs = list(combinations(methods, 2))
method_diffs = []
for pair in method_pairs:
    diff = method_means[pair[1]] - method_means[pair[0]]
    method_diffs.append(diff)

plt.bar(range(len(method_diffs)), method_diffs, color='lightblue')
plt.xticks(range(len(method_diffs)), [f'{pair[0]}-{pair[1]}' for pair in method_pairs], rotation=45)
plt.ylabel('Mean Difference')
plt.title('Post-hoc Comparison - Methods')
plt.grid(True, alpha=0.3)

# Plot 12: Power analysis
plt.subplot(3, 4, 12)
# Simulasi power analysis
effect_sizes = np.linspace(0.1, 1.0, 100)
powers = []
for es in effect_sizes:
    # Simulasi power calculation
    power = 1 - f.cdf(f.ppf(0.95, df_method, df_error), df_method, df_error)
    powers.append(power)

plt.plot(effect_sizes, powers, 'b-', linewidth=2)
plt.axhline(0.8, color='red', linestyle='--', label='Power = 0.8')
plt.xlabel('Effect Size')
plt.ylabel('Power')
plt.title('Power Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 5. Interpretasi Hasil
print("\n5. INTERPRETASI HASIL:")
alpha = 0.05

print(f"\nEfek Utama Method (Faktor A):")
if p_method < alpha:
    print(f"  - Signifikan (p = {p_method:.4f} < {alpha})")
    print("  - Kesimpulan: Method mempengaruhi score")
else:
    print(f"  - Tidak signifikan (p = {p_method:.4f} >= {alpha})")
    print("  - Kesimpulan: Method tidak mempengaruhi score")

print(f"\nEfek Utama Ability (Faktor B):")
if p_ability < alpha:
    print(f"  - Signifikan (p = {p_ability:.4f} < {alpha})")
    print("  - Kesimpulan: Ability mempengaruhi score")
else:
    print(f"  - Tidak signifikan (p = {p_ability:.4f} >= {alpha})")
    print("  - Kesimpulan: Ability tidak mempengaruhi score")

print(f"\nEfek Interaksi Method × Ability:")
if p_interaction < alpha:
    print(f"  - Signifikan (p = {p_interaction:.4f} < {alpha})")
    print("  - Kesimpulan: Ada interaksi antara method dan ability")
    print("  - Rekomendasi: Analisis lebih detail untuk memahami interaksi")
else:
    print(f"  - Tidak signifikan (p = {p_interaction:.4f} >= {alpha})")
    print("  - Kesimpulan: Tidak ada interaksi antara method dan ability")

# 6. Kesimpulan dan Rekomendasi
print("\n6. KESIMPULAN DAN REKOMENDASI:")
print("   - Two-way ANOVA: Uji perbedaan rata-rata dengan 2 faktor")
print("   - Efek utama: Pengaruh individual setiap faktor")
print("   - Efek interaksi: Pengaruh kombinasi faktor")
print("   - Asumsi: Normalitas, homogenitas varian, independensi")
print("   - Effect size: Eta-squared dan partial eta-squared")
print("   - Visualisasi: Interaction plot, heatmap, box plot")
print("   - Post-hoc: Analisis lebih detail jika ada efek signifikan")


## 5. F-test dan F-distribution

### 5.1 Pengertian F-test

**F-test** adalah uji statistik yang menggunakan distribusi F untuk membandingkan varians atau mean squares dari dua atau lebih kelompok. F-test adalah dasar dari ANOVA.

### 5.2 F-distribution

#### 5.2.1 Karakteristik F-distribution
- **Bentuk**: Asimetris, selalu positif
- **Parameter**: df1 (degrees of freedom numerator), df2 (degrees of freedom denominator)
- **Range**: [0, ∞)
- **Mean**: df2/(df2-2) untuk df2 > 2
- **Variance**: 2df2²(df1+df2-2)/(df1(df2-2)²(df2-4)) untuk df2 > 4

#### 5.2.2 Rumus F-distribution
```
F = (SS_between/df_between) / (SS_within/df_within)
```

Dimana:
- **SS_between**: Sum of squares between groups
- **SS_within**: Sum of squares within groups
- **df_between**: Degrees of freedom between groups
- **df_within**: Degrees of freedom within groups

### 5.3 Jenis-jenis F-test

#### 5.3.1 One-way ANOVA F-test
- **Tujuan**: Membandingkan mean dari 3+ kelompok
- **Hipotesis**: H0: μ1 = μ2 = ... = μk
- **F-statistic**: F = MS_between/MS_within

#### 5.3.2 Two-way ANOVA F-test
- **Tujuan**: Membandingkan mean dengan 2 faktor
- **Hipotesis**: H0A, H0B, H0AB
- **F-statistics**: F_A, F_B, F_AB

#### 5.3.3 F-test untuk Homogenitas Varian
- **Tujuan**: Menguji apakah varian sama antar kelompok
- **Hipotesis**: H0: σ1² = σ2² = ... = σk²
- **F-statistic**: F = max(s²)/min(s²)

### 5.4 Interpretasi F-test

#### 5.4.1 F-statistic
- **F > F_critical**: Tolak H0 (ada perbedaan signifikan)
- **F ≤ F_critical**: Gagal tolak H0 (tidak ada perbedaan signifikan)

#### 5.4.2 P-value
- **p < α**: Tolak H0 (signifikan)
- **p ≥ α**: Gagal tolak H0 (tidak signifikan)

#### 5.4.3 Effect Size
- **Eta-squared (η²)**: SS_between/SS_total
- **Partial Eta-squared**: SS_effect/(SS_effect + SS_error)
- **Cohen's f**: √(η²/(1-η²))

### 5.5 Aplikasi F-test

1. **Penelitian Eksperimental**: Membandingkan perlakuan
2. **Penelitian Observasional**: Membandingkan kelompok
3. **Quality Control**: Membandingkan proses
4. **A/B Testing**: Membandingkan versi
5. **Penelitian Medis**: Membandingkan pengobatan


In [ ]:
# Demonstrasi F-test dan F-distribution
print("=== DEMONSTRASI F-TEST DAN F-DISTRIBUTION ===")

# Data untuk demonstrasi F-test
np.random.seed(42)

# Simulasi data dengan 4 kelompok
group1 = np.random.normal(50, 10, 30)
group2 = np.random.normal(55, 10, 30)
group3 = np.random.normal(60, 10, 30)
group4 = np.random.normal(52, 10, 30)

print("Data untuk F-test:")
print(f"Group 1: mean={np.mean(group1):.2f}, std={np.std(group1, ddof=1):.2f}")
print(f"Group 2: mean={np.mean(group2):.2f}, std={np.std(group2, ddof=1):.2f}")
print(f"Group 3: mean={np.mean(group3):.2f}, std={np.std(group3, ddof=1):.2f}")
print(f"Group 4: mean={np.mean(group4):.2f}, std={np.std(group4, ddof=1):.2f}")

# 1. F-test untuk One-way ANOVA
print("\n1. F-TEST UNTUK ONE-WAY ANOVA:")

# Hitung F-statistic
all_data = np.concatenate([group1, group2, group3, group4])
grand_mean = np.mean(all_data)
group_means = [np.mean(group1), np.mean(group2), np.mean(group3), np.mean(group4)]
group_sizes = [len(group1), len(group2), len(group3), len(group4)]

# SS_between
ss_between = sum([n * (mean - grand_mean)**2 for n, mean in zip(group_sizes, group_means)])
print(f"SS_between: {ss_between:.2f}")

# SS_within
ss_within = sum([sum((group - np.mean(group))**2) for group in [group1, group2, group3, group4]])
print(f"SS_within: {ss_within:.2f}")

# Degrees of freedom
k = 4  # jumlah kelompok
N = len(all_data)
df_between = k - 1
df_within = N - k
print(f"df_between: {df_between}")
print(f"df_within: {df_within}")

# Mean Squares
ms_between = ss_between / df_between
ms_within = ss_within / df_within
print(f"MS_between: {ms_between:.2f}")
print(f"MS_within: {ms_within:.2f}")

# F-statistic
f_statistic = ms_between / ms_within
print(f"F-statistic: {f_statistic:.4f}")

# P-value
p_value = 1 - f.cdf(f_statistic, df_between, df_within)
print(f"p-value: {p_value:.4f}")

# F-critical
f_critical = f.ppf(0.95, df_between, df_within)
print(f"F-critical (α=0.05): {f_critical:.4f}")

# 2. F-test untuk Homogenitas Varian
print("\n2. F-TEST UNTUK HOMOGENITAS VARIAN:")

# Hitung varian setiap kelompok
var1 = np.var(group1, ddof=1)
var2 = np.var(group2, ddof=1)
var3 = np.var(group3, ddof=1)
var4 = np.var(group4, ddof=1)

print(f"Varian Group 1: {var1:.2f}")
print(f"Varian Group 2: {var2:.2f}")
print(f"Varian Group 3: {var3:.2f}")
print(f"Varian Group 4: {var4:.2f}")

# F-test untuk homogenitas
max_var = max(var1, var2, var3, var4)
min_var = min(var1, var2, var3, var4)
f_homogeneity = max_var / min_var
print(f"F-test (max/min): {f_homogeneity:.4f}")

# 3. Effect Size
print("\n3. EFFECT SIZE:")

# Eta-squared
ss_total = sum((all_data - grand_mean)**2)
eta_squared = ss_between / ss_total
print(f"Eta-squared (η²): {eta_squared:.4f}")

# Partial eta-squared
partial_eta_squared = ss_between / (ss_between + ss_within)
print(f"Partial Eta-squared: {partial_eta_squared:.4f}")

# Cohen's f
cohens_f = np.sqrt(eta_squared / (1 - eta_squared))
print(f"Cohen's f: {cohens_f:.4f}")

# 4. Visualisasi F-test dan F-distribution
plt.figure(figsize=(20, 15))

# Plot 1: F-distribution
plt.subplot(3, 4, 1)
x = np.linspace(0, 10, 100)
f_dist = f.pdf(x, df_between, df_within)
plt.plot(x, f_dist, 'b-', linewidth=2, label=f'F-distribution (df1={df_between}, df2={df_within})')
plt.axvline(f_statistic, color='red', linestyle='--', linewidth=2, label=f'F-statistic: {f_statistic:.2f}')
plt.axvline(f_critical, color='green', linestyle=':', linewidth=2, label=f'F-critical: {f_critical:.2f}')
plt.xlabel('F-value')
plt.ylabel('Density')
plt.title('F-distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: F-distribution dengan berbagai df
plt.subplot(3, 4, 2)
x = np.linspace(0, 5, 100)
for df1, df2 in [(2, 10), (5, 10), (10, 10), (10, 5)]:
    f_dist = f.pdf(x, df1, df2)
    plt.plot(x, f_dist, linewidth=2, label=f'df1={df1}, df2={df2}')
plt.xlabel('F-value')
plt.ylabel('Density')
plt.title('F-distribution dengan Berbagai df')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Cumulative F-distribution
plt.subplot(3, 4, 3)
x = np.linspace(0, 10, 100)
f_cdf = f.cdf(x, df_between, df_within)
plt.plot(x, f_cdf, 'b-', linewidth=2, label=f'F-CDF (df1={df_between}, df2={df_within})')
plt.axvline(f_statistic, color='red', linestyle='--', linewidth=2, label=f'F-statistic: {f_statistic:.2f}')
plt.axhline(0.95, color='green', linestyle=':', linewidth=2, label='95% quantile')
plt.xlabel('F-value')
plt.ylabel('Cumulative Probability')
plt.title('Cumulative F-distribution')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 4: P-value visualization
plt.subplot(3, 4, 4)
x = np.linspace(0, 10, 100)
f_dist = f.pdf(x, df_between, df_within)
plt.plot(x, f_dist, 'b-', linewidth=2, label=f'F-distribution')
plt.fill_between(x[x >= f_statistic], f_dist[x >= f_statistic], alpha=0.3, color='red', label=f'p-value: {p_value:.4f}')
plt.axvline(f_statistic, color='red', linestyle='--', linewidth=2, label=f'F-statistic: {f_statistic:.2f}')
plt.xlabel('F-value')
plt.ylabel('Density')
plt.title('P-value Visualization')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 5: Group means comparison
plt.subplot(3, 4, 5)
means = [np.mean(group1), np.mean(group2), np.mean(group3), np.mean(group4)]
groups = ['Group 1', 'Group 2', 'Group 3', 'Group 4']
bars = plt.bar(groups, means, alpha=0.7, color=['blue', 'green', 'red', 'orange'])
plt.ylabel('Mean Value')
plt.title('Group Means Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Add mean values on bars
for bar, mean in zip(bars, means):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
             f'{mean:.2f}', ha='center', va='bottom')

# Plot 6: Variance comparison
plt.subplot(3, 4, 6)
variances = [var1, var2, var3, var4]
plt.bar(groups, variances, alpha=0.7, color=['blue', 'green', 'red', 'orange'])
plt.ylabel('Variance')
plt.title('Variance Comparison')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 7: Box plot
plt.subplot(3, 4, 7)
data_for_box = [group1, group2, group3, group4]
box_plot = plt.boxplot(data_for_box, labels=groups, patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
plt.ylabel('Values')
plt.title('Box Plot')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 8: Histogram per kelompok
plt.subplot(3, 4, 8)
plt.hist(group1, alpha=0.5, label='Group 1', bins=10, color='blue')
plt.hist(group2, alpha=0.5, label='Group 2', bins=10, color='green')
plt.hist(group3, alpha=0.5, label='Group 3', bins=10, color='red')
plt.hist(group4, alpha=0.5, label='Group 4', bins=10, color='orange')
plt.xlabel('Values')
plt.ylabel('Frequency')
plt.title('Histogram per Kelompok')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 9: Effect size comparison
plt.subplot(3, 4, 9)
effect_sizes = [eta_squared, partial_eta_squared, cohens_f**2]
effect_names = ['η²', 'Partial η²', 'Cohen\'s f²']
plt.bar(effect_names, effect_sizes, alpha=0.7, color=['purple', 'brown', 'pink'])
plt.ylabel('Effect Size')
plt.title('Effect Size Comparison')
plt.grid(True, alpha=0.3)

# Plot 10: Power analysis
plt.subplot(3, 4, 10)
effect_sizes = np.linspace(0.1, 1.0, 100)
powers = []
for es in effect_sizes:
    # Simulasi power calculation
    power = 1 - f.cdf(f.ppf(0.95, df_between, df_within), df_between, df_within)
    powers.append(power)

plt.plot(effect_sizes, powers, 'b-', linewidth=2)
plt.axhline(0.8, color='red', linestyle='--', label='Power = 0.8')
plt.xlabel('Effect Size')
plt.ylabel('Power')
plt.title('Power Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 11: F-test summary
plt.subplot(3, 4, 11)
plt.axis('off')
summary_text = f"""
F-TEST SUMMARY

F-statistic: {f_statistic:.4f}
p-value: {p_value:.4f}
F-critical: {f_critical:.4f}

Degrees of Freedom:
Between: {df_between}
Within: {df_within}

Effect Sizes:
η²: {eta_squared:.4f}
Partial η²: {partial_eta_squared:.4f}
Cohen's f: {cohens_f:.4f}

Decision: {'Reject H0' if p_value < 0.05 else 'Fail to reject H0'}
"""
plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, fontsize=9, 
         verticalalignment='top', fontfamily='monospace')

# Plot 12: Homogeneity test
plt.subplot(3, 4, 12)
plt.bar(groups, variances, alpha=0.7, color=['blue', 'green', 'red', 'orange'])
plt.axhline(min_var, color='green', linestyle='--', label=f'Min: {min_var:.2f}')
plt.axhline(max_var, color='red', linestyle='--', label=f'Max: {max_var:.2f}')
plt.ylabel('Variance')
plt.title('Homogeneity Test')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# 5. Interpretasi Hasil
print("\n5. INTERPRETASI HASIL:")
alpha = 0.05

print(f"\nF-test untuk One-way ANOVA:")
if p_value < alpha:
    print(f"  - Tolak H0 (p = {p_value:.4f} < {alpha})")
    print("  - Kesimpulan: Ada perbedaan signifikan antar kelompok")
else:
    print(f"  - Gagal tolak H0 (p = {p_value:.4f} >= {alpha})")
    print("  - Kesimpulan: Tidak ada perbedaan signifikan antar kelompok")

print(f"\nF-test untuk Homogenitas Varian:")
if f_homogeneity < 2:  # Rule of thumb
    print(f"  - Varian homogen (F = {f_homogeneity:.4f} < 2)")
    print("  - Asumsi homogenitas terpenuhi")
else:
    print(f"  - Varian heterogen (F = {f_homogeneity:.4f} >= 2)")
    print("  - Asumsi homogenitas dilanggar")

print(f"\nEffect Size:")
if eta_squared < 0.01:
    effect_size = "Negligible"
elif eta_squared < 0.06:
    effect_size = "Small"
elif eta_squared < 0.14:
    effect_size = "Medium"
else:
    effect_size = "Large"

print(f"  - Effect size: {effect_size} (η² = {eta_squared:.4f})")

# 6. Kesimpulan dan Rekomendasi
print("\n6. KESIMPULAN DAN REKOMENDASI:")
print("   - F-test: Dasar dari ANOVA untuk membandingkan varians")
print("   - F-distribution: Distribusi sampling untuk F-statistic")
print("   - Interpretasi: F-statistic, p-value, dan effect size")
print("   - Asumsi: Normalitas, homogenitas varian, independensi")
print("   - Power analysis: Penting untuk perencanaan penelitian")
print("   - Visualisasi: F-distribution, group means, effect sizes")


## 6. Post-hoc Analysis (Tukey HSD, Bonferroni)

### 6.1 Pengertian Post-hoc Analysis

**Post-hoc Analysis** adalah analisis lanjutan yang dilakukan setelah ANOVA menunjukkan hasil signifikan. Tujuannya adalah untuk mengetahui kelompok mana yang berbeda secara signifikan.

### 6.2 Kapan Menggunakan Post-hoc Analysis

- **ANOVA signifikan**: F-test menunjukkan p < α
- **Multiple comparisons**: Membandingkan semua pasangan kelompok
- **Family-wise error rate**: Mengontrol error rate untuk multiple testing
- **Specific comparisons**: Menguji perbedaan spesifik antar kelompok

### 6.3 Jenis-jenis Post-hoc Tests

#### 6.3.1 Tukey HSD (Honestly Significant Difference)
- **Tujuan**: Membandingkan semua pasangan kelompok
- **Keunggulan**: Mengontrol family-wise error rate
- **Kekurangan**: Konservatif untuk banyak kelompok
- **Formula**: HSD = q_α,k,df × √(MS_within/n)

#### 6.3.2 Bonferroni Correction
- **Tujuan**: Mengontrol family-wise error rate
- **Keunggulan**: Sederhana dan mudah dipahami
- **Kekurangan**: Sangat konservatif
- **Formula**: α_adjusted = α/k

#### 6.3.3 Scheffe Test
- **Tujuan**: Membandingkan semua kombinasi kelompok
- **Keunggulan**: Fleksibel untuk berbagai perbandingan
- **Kekurangan**: Kurang powerful untuk pairwise comparisons
- **Formula**: F = (MS_contrast)/(MS_within)

#### 6.3.4 Fisher's LSD (Least Significant Difference)
- **Tujuan**: Pairwise comparisons setelah ANOVA signifikan
- **Keunggulan**: Powerful untuk pairwise comparisons
- **Kekurangan**: Tidak mengontrol family-wise error rate
- **Formula**: LSD = t_α/2,df × √(2×MS_within/n)

### 6.4 Interpretasi Post-hoc Results

#### 6.4.1 Significant Differences
- **p < α**: Kelompok berbeda signifikan
- **p ≥ α**: Kelompok tidak berbeda signifikan
- **Effect size**: Besarnya perbedaan antar kelompok

#### 6.4.2 Effect Size untuk Post-hoc
- **Cohen's d**: (μ1 - μ2)/σ_pooled
- **Glass's Δ**: (μ1 - μ2)/σ_control
- **Hedges' g**: Cohen's d dengan correction factor

### 6.5 Aplikasi Post-hoc Analysis

1. **Penelitian Eksperimental**: Membandingkan perlakuan
2. **Penelitian Observasional**: Membandingkan kelompok
3. **Quality Control**: Membandingkan proses
4. **A/B Testing**: Membandingkan versi
5. **Penelitian Medis**: Membandingkan pengobatan

### 6.6 Best Practices

1. **Pilih test yang tepat**: Berdasarkan tujuan dan asumsi
2. **Kontrol error rate**: Gunakan correction untuk multiple testing
3. **Report effect sizes**: Jangan hanya p-value
4. **Visualisasi**: Gunakan grafik untuk memahami perbedaan
5. **Interpretasi hati-hati**: Pertimbangkan konteks penelitian


In [ ]:
# Demonstrasi Post-hoc Analysis
print("=== DEMONSTRASI POST-HOC ANALYSIS ===")

# Data untuk demonstrasi post-hoc analysis
np.random.seed(42)

# Simulasi data dengan 4 kelompok
group1 = np.random.normal(50, 10, 30)
group2 = np.random.normal(55, 10, 30)
group3 = np.random.normal(60, 10, 30)
group4 = np.random.normal(52, 10, 30)

print("Data untuk Post-hoc Analysis:")
print(f"Group 1: mean={np.mean(group1):.2f}, std={np.std(group1, ddof=1):.2f}")
print(f"Group 2: mean={np.mean(group2):.2f}, std={np.std(group2, ddof=1):.2f}")
print(f"Group 3: mean={np.mean(group3):.2f}, std={np.std(group3, ddof=1):.2f}")
print(f"Group 4: mean={np.mean(group4):.2f}, std={np.std(group4, ddof=1):.2f}")

# 1. Tukey HSD Test
print("\n1. TUKEY HSD TEST:")

# Gabungkan data
all_data = np.concatenate([group1, group2, group3, group4])
groups = ['Group1'] * len(group1) + ['Group2'] * len(group2) + ['Group3'] * len(group3) + ['Group4'] * len(group4)

# Tukey HSD test
from statsmodels.stats.multicomp import pairwise_tukeyhsd
tukey_result = pairwise_tukeyhsd(endog=all_data, groups=groups, alpha=0.05)
print("Tukey HSD Results:")
print(tukey_result)

# 2. Bonferroni Correction
print("\n2. BONFERRONI CORRECTION:")

# Hitung jumlah perbandingan
k = 4  # jumlah kelompok
n_comparisons = k * (k - 1) // 2
print(f"Jumlah perbandingan: {n_comparisons}")

# Alpha adjusted
alpha = 0.05
alpha_adjusted = alpha / n_comparisons
print(f"Alpha adjusted: {alpha_adjusted:.4f}")

# T-test untuk setiap pasangan dengan Bonferroni correction
from scipy.stats import ttest_ind
from itertools import combinations

print("\nPairwise t-tests dengan Bonferroni correction:")
group_names = ['Group1', 'Group2', 'Group3', 'Group4']
group_data = [group1, group2, group3, group4]

for i, (group1_name, group2_name) in enumerate(combinations(group_names, 2)):
    group1_data = group_data[group_names.index(group1_name)]
    group2_data = group_data[group_names.index(group2_name)]
    
    t_stat, p_value = ttest_ind(group1_data, group2_data)
    p_value_adjusted = p_value * n_comparisons
    p_value_adjusted = min(p_value_adjusted, 1.0)  # Cap at 1.0
    
    print(f"  {group1_name} vs {group2_name}:")
    print(f"    t-statistic: {t_stat:.4f}")
    print(f"    p-value: {p_value:.4f}")
    print(f"    p-value (adjusted): {p_value_adjusted:.4f}")
    print(f"    Significant: {'Yes' if p_value_adjusted < alpha else 'No'}")

# 3. Effect Size untuk Post-hoc
print("\n3. EFFECT SIZE UNTUK POST-HOC:")

# Cohen's d untuk setiap pasangan
def cohens_d(group1, group2):
    n1, n2 = len(group1), len(group2)
    s1, s2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    pooled_std = np.sqrt(((n1-1)*s1**2 + (n2-1)*s2**2) / (n1+n2-2))
    return (np.mean(group1) - np.mean(group2)) / pooled_std

print("Cohen's d untuk setiap pasangan:")
for i, (group1_name, group2_name) in enumerate(combinations(group_names, 2)):
    group1_data = group_data[group_names.index(group1_name)]
    group2_data = group_data[group_names.index(group2_name)]
    
    d = cohens_d(group1_data, group2_data)
    print(f"  {group1_name} vs {group2_name}: d = {d:.4f}")

# 4. Visualisasi Post-hoc Analysis
plt.figure(figsize=(20, 15))

# Plot 1: Tukey HSD results
plt.subplot(3, 4, 1)
# Simulasi hasil Tukey HSD
tukey_means = [np.mean(group1), np.mean(group2), np.mean(group3), np.mean(group4)]
tukey_errors = [np.std(group, ddof=1)/np.sqrt(len(group)) for group in [group1, group2, group3, group4]]
plt.errorbar(group_names, tukey_means, yerr=tukey_errors, fmt='o', capsize=5, capthick=2, 
             color='blue', markersize=8)
plt.ylabel('Mean ± SE')
plt.title('Tukey HSD - Group Means')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 2: Pairwise comparisons
plt.subplot(3, 4, 2)
# Simulasi hasil pairwise comparisons
comparisons = [f'{pair[0]}-{pair[1]}' for pair in combinations(group_names, 2)]
p_values = [0.001, 0.023, 0.045, 0.067, 0.089, 0.112]  # Simulasi
p_values_adjusted = [p * n_comparisons for p in p_values]
p_values_adjusted = [min(p, 1.0) for p in p_values_adjusted]

plt.bar(range(len(comparisons)), p_values, alpha=0.7, color='lightblue', label='Original p-values')
plt.bar(range(len(comparisons)), p_values_adjusted, alpha=0.7, color='lightcoral', label='Adjusted p-values')
plt.axhline(alpha, color='red', linestyle='--', label=f'α = {alpha}')
plt.xticks(range(len(comparisons)), comparisons, rotation=45)
plt.ylabel('P-value')
plt.title('Pairwise Comparisons')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 3: Effect sizes
plt.subplot(3, 4, 3)
effect_sizes = [cohens_d(group_data[group_names.index(pair[0])], group_data[group_names.index(pair[1])]) 
                for pair in combinations(group_names, 2)]
plt.bar(range(len(comparisons)), effect_sizes, alpha=0.7, color='lightgreen')
plt.xticks(range(len(comparisons)), comparisons, rotation=45)
plt.ylabel("Cohen's d")
plt.title('Effect Sizes')
plt.grid(True, alpha=0.3)

# Plot 4: Box plot dengan significant differences
plt.subplot(3, 4, 4)
data_for_box = [group1, group2, group3, group4]
box_plot = plt.boxplot(data_for_box, labels=group_names, patch_artist=True)
colors = ['lightblue', 'lightgreen', 'lightcoral', 'lightyellow']
for patch, color in zip(box_plot['boxes'], colors):
    patch.set_facecolor(color)
plt.ylabel('Values')
plt.title('Box Plot dengan Significant Differences')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 5: Mean comparison dengan confidence intervals
plt.subplot(3, 4, 5)
means = [np.mean(group) for group in [group1, group2, group3, group4]]
stds = [np.std(group, ddof=1) for group in [group1, group2, group3, group4]]
n = len(group1)
se = [std/np.sqrt(n) for std in stds]
ci = [1.96 * se_val for se_val in se]  # 95% CI

plt.errorbar(group_names, means, yerr=ci, fmt='o', capsize=5, capthick=2, 
             color='blue', markersize=8)
plt.ylabel('Mean ± 95% CI')
plt.title('Group Means dengan Confidence Intervals')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 6: Multiple comparison correction
plt.subplot(3, 4, 6)
corrections = ['None', 'Bonferroni', 'Holm', 'FDR']
alpha_original = 0.05
alpha_corrected = [alpha_original, alpha_original/n_comparisons, alpha_original/n_comparisons, alpha_original]
plt.bar(corrections, alpha_corrected, alpha=0.7, color=['red', 'blue', 'green', 'orange'])
plt.ylabel('Alpha Level')
plt.title('Multiple Comparison Corrections')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 7: Power analysis untuk post-hoc
plt.subplot(3, 4, 7)
effect_sizes = np.linspace(0.1, 1.0, 100)
powers = []
for es in effect_sizes:
    # Simulasi power calculation
    power = 1 - f.cdf(f.ppf(0.95, 3, 116), 3, 116)  # df_between=3, df_within=116
    powers.append(power)

plt.plot(effect_sizes, powers, 'b-', linewidth=2)
plt.axhline(0.8, color='red', linestyle='--', label='Power = 0.8')
plt.xlabel('Effect Size')
plt.ylabel('Power')
plt.title('Power Analysis untuk Post-hoc')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 8: Family-wise error rate
plt.subplot(3, 4, 8)
n_tests = range(1, 21)
fwer = [1 - (1 - alpha)**n for n in n_tests]
plt.plot(n_tests, fwer, 'b-', linewidth=2, label='No correction')
plt.plot(n_tests, [alpha] * len(n_tests), 'r--', linewidth=2, label=f'α = {alpha}')
plt.xlabel('Number of Tests')
plt.ylabel('Family-wise Error Rate')
plt.title('Family-wise Error Rate')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 9: Post-hoc summary
plt.subplot(3, 4, 9)
plt.axis('off')
summary_text = f"""
POST-HOC ANALYSIS SUMMARY

Tukey HSD: Available
Bonferroni: α = {alpha_adjusted:.4f}
Comparisons: {n_comparisons}

Effect Sizes (Cohen's d):
Group1 vs Group2: {cohens_d(group1, group2):.4f}
Group1 vs Group3: {cohens_d(group1, group3):.4f}
Group1 vs Group4: {cohens_d(group1, group4):.4f}
Group2 vs Group3: {cohens_d(group2, group3):.4f}
Group2 vs Group4: {cohens_d(group2, group4):.4f}
Group3 vs Group4: {cohens_d(group3, group4):.4f}

Recommendations:
- Use Tukey HSD for all pairwise comparisons
- Report effect sizes with p-values
- Consider practical significance
"""
plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, fontsize=8, 
         verticalalignment='top', fontfamily='monospace')

# Plot 10: Significant differences heatmap
plt.subplot(3, 4, 10)
# Simulasi heatmap untuk significant differences
heatmap_data = np.zeros((4, 4))
for i in range(4):
    for j in range(4):
        if i != j:
            # Simulasi significant differences
            if abs(i - j) == 1:  # Adjacent groups
                heatmap_data[i, j] = 1
            elif abs(i - j) == 2:  # Two groups apart
                heatmap_data[i, j] = 0.5
            else:  # Three groups apart
                heatmap_data[i, j] = 0

sns.heatmap(heatmap_data, annot=True, fmt='.1f', cmap='RdYlBu_r', 
            xticklabels=group_names, yticklabels=group_names, ax=plt.gca())
plt.title('Significant Differences Heatmap')
plt.xlabel('Group')
plt.ylabel('Group')

# Plot 11: Multiple testing problem
plt.subplot(3, 4, 11)
n_tests = range(1, 11)
alpha_original = 0.05
alpha_bonferroni = [alpha_original / n for n in n_tests]
alpha_holm = [alpha_original / n for n in n_tests]

plt.plot(n_tests, [alpha_original] * len(n_tests), 'r-', linewidth=2, label='No correction')
plt.plot(n_tests, alpha_bonferroni, 'b-', linewidth=2, label='Bonferroni')
plt.plot(n_tests, alpha_holm, 'g-', linewidth=2, label='Holm')
plt.xlabel('Number of Tests')
plt.ylabel('Alpha Level')
plt.title('Multiple Testing Problem')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 12: Post-hoc recommendations
plt.subplot(3, 4, 12)
plt.axis('off')
recommendations_text = f"""
POST-HOC RECOMMENDATIONS

1. Choose appropriate test:
   - Tukey HSD: All pairwise comparisons
   - Bonferroni: Few comparisons
   - Scheffe: Complex comparisons
   - Fisher's LSD: Planned comparisons

2. Control error rate:
   - Family-wise error rate
   - False discovery rate
   - Per-comparison error rate

3. Report effect sizes:
   - Cohen's d
   - Glass's Δ
   - Hedges' g

4. Consider practical significance:
   - Effect size interpretation
   - Clinical significance
   - Business impact
"""
plt.text(0.1, 0.9, recommendations_text, transform=plt.gca().transAxes, fontsize=8, 
         verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# 5. Interpretasi Hasil
print("\n5. INTERPRETASI HASIL:")

print(f"\nTukey HSD Test:")
print("  - Mengontrol family-wise error rate")
print("  - Membandingkan semua pasangan kelompok")
print("  - Hasil: Lihat output di atas")

print(f"\nBonferroni Correction:")
print(f"  - Alpha adjusted: {alpha_adjusted:.4f}")
print(f"  - Jumlah perbandingan: {n_comparisons}")
print("  - Hasil: Lihat output di atas")

print(f"\nEffect Sizes:")
print("  - Cohen's d untuk setiap pasangan")
print("  - Interpretasi: 0.2 (small), 0.5 (medium), 0.8 (large)")

# 6. Kesimpulan dan Rekomendasi
print("\n6. KESIMPULAN DAN REKOMENDASI:")
print("   - Post-hoc analysis: Analisis lanjutan setelah ANOVA signifikan")
print("   - Tukey HSD: Pilihan terbaik untuk semua pairwise comparisons")
print("   - Bonferroni: Sederhana tapi konservatif")
print("   - Effect sizes: Penting untuk interpretasi praktis")
print("   - Multiple testing: Kontrol error rate dengan hati-hati")
print("   - Visualisasi: Gunakan grafik untuk memahami perbedaan")


## 7. Aplikasi Praktis ANOVA

### 7.1 Penelitian Eksperimental

#### 7.1.1 A/B Testing
- **Tujuan**: Membandingkan efektivitas dua versi
- **Metode**: One-way ANOVA dengan 2 kelompok
- **Contoh**: Website A vs Website B untuk conversion rate

#### 7.1.2 Multi-variant Testing
- **Tujuan**: Membandingkan beberapa versi sekaligus
- **Metode**: One-way ANOVA dengan 3+ kelompok
- **Contoh**: 4 desain landing page untuk click-through rate

#### 7.1.3 Controlled Experiments
- **Tujuan**: Menguji efek perlakuan dengan kontrol
- **Metode**: One-way ANOVA dengan kelompok kontrol
- **Contoh**: Pengaruh metode pembelajaran terhadap nilai siswa

### 7.2 Penelitian Observasional

#### 7.2.1 Cross-sectional Studies
- **Tujuan**: Membandingkan kelompok pada satu waktu
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan pendapatan berdasarkan tingkat pendidikan

#### 7.2.2 Longitudinal Studies
- **Tujuan**: Membandingkan kelompok sepanjang waktu
- **Metode**: Two-way ANOVA (kelompok × waktu)
- **Contoh**: Perubahan performa karyawan berdasarkan departemen dan tahun

#### 7.2.3 Cohort Studies
- **Tujuan**: Membandingkan kelompok dengan karakteristik berbeda
- **Metode**: One-way atau Two-way ANOVA
- **Contoh**: Perbedaan kesehatan berdasarkan usia dan jenis kelamin

### 7.3 Quality Control

#### 7.3.1 Process Comparison
- **Tujuan**: Membandingkan kualitas proses produksi
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan kualitas produk dari 3 mesin produksi

#### 7.3.2 Supplier Evaluation
- **Tujuan**: Membandingkan performa supplier
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan kualitas material dari 5 supplier

#### 7.3.3 Equipment Performance
- **Tujuan**: Membandingkan performa peralatan
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan efisiensi 4 mesin pengemasan

### 7.4 Penelitian Medis

#### 7.4.1 Clinical Trials
- **Tujuan**: Membandingkan efektivitas pengobatan
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan recovery time untuk 3 jenis obat

#### 7.4.2 Treatment Comparison
- **Tujuan**: Membandingkan berbagai metode pengobatan
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan efektivitas terapi fisik, obat, dan kombinasi

#### 7.4.3 Patient Groups
- **Tujuan**: Membandingkan respons pasien
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan respons pengobatan berdasarkan usia pasien

### 7.5 Penelitian Psikologi

#### 7.5.1 Experimental Psychology
- **Tujuan**: Menguji efek eksperimen
- **Metode**: One-way ANOVA
- **Contoh**: Pengaruh musik terhadap performa kognitif

#### 7.5.2 Social Psychology
- **Tujuan**: Membandingkan kelompok sosial
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan sikap berdasarkan kelompok usia

#### 7.5.3 Educational Psychology
- **Tujuan**: Membandingkan metode pembelajaran
- **Metode**: One-way ANOVA
- **Contoh**: Efektivitas berbagai metode mengajar

### 7.6 Penelitian Bisnis

#### 7.6.1 Marketing Research
- **Tujuan**: Membandingkan strategi pemasaran
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan penjualan berdasarkan jenis iklan

#### 7.6.2 Customer Segmentation
- **Tujuan**: Membandingkan kelompok pelanggan
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan spending berdasarkan segmentasi demografis

#### 7.6.3 Product Development
- **Tujuan**: Membandingkan versi produk
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan kepuasan pelanggan untuk 3 versi produk

### 7.7 Penelitian Pertanian

#### 7.7.1 Crop Yield
- **Tujuan**: Membandingkan hasil panen
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan hasil panen untuk 4 jenis pupuk

#### 7.7.2 Soil Treatment
- **Tujuan**: Membandingkan perlakuan tanah
- **Metode**: One-way ANOVA
- **Contoh**: Pengaruh pH tanah terhadap pertumbuhan tanaman

#### 7.7.3 Irrigation Methods
- **Tujuan**: Membandingkan metode irigasi
- **Metode**: One-way ANOVA
- **Contoh**: Efektivitas 3 sistem irigasi untuk produksi padi

### 7.8 Penelitian Lingkungan

#### 7.8.1 Pollution Studies
- **Tujuan**: Membandingkan tingkat polusi
- **Metode**: One-way ANOVA
- **Contoh**: Perbedaan kualitas air di 5 lokasi sungai

#### 7.8.2 Climate Impact
- **Tujuan**: Membandingkan dampak iklim
- **Metode**: One-way ANOVA
- **Contoh**: Pengaruh suhu terhadap pertumbuhan hutan

#### 7.8.3 Conservation Methods
- **Tujuan**: Membandingkan metode konservasi
- **Metode**: One-way ANOVA
- **Contoh**: Efektivitas 3 program konservasi satwa liar


In [ ]:
# Demonstrasi Aplikasi Praktis ANOVA
print("=== DEMONSTRASI APLIKASI PRAKTIS ANOVA ===")

# Simulasi data untuk berbagai aplikasi praktis
np.random.seed(42)

# 1. A/B Testing - Website Conversion Rate
print("1. A/B TESTING - WEBSITE CONVERSION RATE:")

# Simulasi data conversion rate untuk 3 versi website
website_a = np.random.normal(0.15, 0.02, 1000)  # 15% conversion rate
website_b = np.random.normal(0.18, 0.02, 1000)  # 18% conversion rate
website_c = np.random.normal(0.16, 0.02, 1000)  # 16% conversion rate

print(f"Website A: mean={np.mean(website_a):.4f}, std={np.std(website_a, ddof=1):.4f}")
print(f"Website B: mean={np.mean(website_b):.4f}, std={np.std(website_b, ddof=1):.4f}")
print(f"Website C: mean={np.mean(website_c):.4f}, std={np.std(website_c, ddof=1):.4f}")

# One-way ANOVA untuk A/B testing
f_stat_ab, p_value_ab = f_oneway(website_a, website_b, website_c)
print(f"F-statistic: {f_stat_ab:.4f}")
print(f"p-value: {p_value_ab:.4f}")

# 2. Quality Control - Product Quality dari 4 Mesin
print("\n2. QUALITY CONTROL - PRODUCT QUALITY DARI 4 MESIN:")

# Simulasi data kualitas produk dari 4 mesin
machine1 = np.random.normal(95, 3, 50)  # Mesin 1: kualitas tinggi
machine2 = np.random.normal(92, 3, 50)  # Mesin 2: kualitas sedang
machine3 = np.random.normal(90, 3, 50)  # Mesin 3: kualitas rendah
machine4 = np.random.normal(94, 3, 50)  # Mesin 4: kualitas tinggi

print(f"Machine 1: mean={np.mean(machine1):.2f}, std={np.std(machine1, ddof=1):.2f}")
print(f"Machine 2: mean={np.mean(machine2):.2f}, std={np.std(machine2, ddof=1):.2f}")
print(f"Machine 3: mean={np.mean(machine3):.2f}, std={np.std(machine3, ddof=1):.2f}")
print(f"Machine 4: mean={np.mean(machine4):.2f}, std={np.std(machine4, ddof=1):.2f}")

# One-way ANOVA untuk quality control
f_stat_qc, p_value_qc = f_oneway(machine1, machine2, machine3, machine4)
print(f"F-statistic: {f_stat_qc:.4f}")
print(f"p-value: {p_value_qc:.4f}")

# 3. Clinical Trial - Recovery Time untuk 3 Jenis Obat
print("\n3. CLINICAL TRIAL - RECOVERY TIME UNTUK 3 JENIS OBAT:")

# Simulasi data recovery time untuk 3 jenis obat
drug_a = np.random.normal(7, 1.5, 30)  # Obat A: 7 hari recovery
drug_b = np.random.normal(5, 1.5, 30)  # Obat B: 5 hari recovery
drug_c = np.random.normal(6, 1.5, 30)  # Obat C: 6 hari recovery

print(f"Drug A: mean={np.mean(drug_a):.2f}, std={np.std(drug_a, ddof=1):.2f}")
print(f"Drug B: mean={np.mean(drug_b):.2f}, std={np.std(drug_b, ddof=1):.2f}")
print(f"Drug C: mean={np.mean(drug_c):.2f}, std={np.std(drug_c, ddof=1):.2f}")

# One-way ANOVA untuk clinical trial
f_stat_ct, p_value_ct = f_oneway(drug_a, drug_b, drug_c)
print(f"F-statistic: {f_stat_ct:.4f}")
print(f"p-value: {p_value_ct:.4f}")

# 4. Educational Research - Metode Pembelajaran
print("\n4. EDUCATIONAL RESEARCH - METODE PEMBELAJARAN:")

# Simulasi data nilai siswa untuk 3 metode pembelajaran
method1 = np.random.normal(75, 8, 40)  # Metode 1: nilai rata-rata
method2 = np.random.normal(80, 8, 40)  # Metode 2: nilai tinggi
method3 = np.random.normal(72, 8, 40)  # Metode 3: nilai rendah

print(f"Method 1: mean={np.mean(method1):.2f}, std={np.std(method1, ddof=1):.2f}")
print(f"Method 2: mean={np.mean(method2):.2f}, std={np.std(method2, ddof=1):.2f}")
print(f"Method 3: mean={np.mean(method3):.2f}, std={np.std(method3, ddof=1):.2f}")

# One-way ANOVA untuk educational research
f_stat_ed, p_value_ed = f_oneway(method1, method2, method3)
print(f"F-statistic: {f_stat_ed:.4f}")
print(f"p-value: {p_value_ed:.4f}")

# 5. Agricultural Research - Hasil Panen untuk 4 Jenis Pupuk
print("\n5. AGRICULTURAL RESEARCH - HASIL PANEN UNTUK 4 JENIS PUPUK:")

# Simulasi data hasil panen untuk 4 jenis pupuk
fertilizer1 = np.random.normal(120, 15, 25)  # Pupuk 1: hasil tinggi
fertilizer2 = np.random.normal(110, 15, 25)  # Pupuk 2: hasil sedang
fertilizer3 = np.random.normal(105, 15, 25)  # Pupuk 3: hasil rendah
fertilizer4 = np.random.normal(115, 15, 25)  # Pupuk 4: hasil tinggi

print(f"Fertilizer 1: mean={np.mean(fertilizer1):.2f}, std={np.std(fertilizer1, ddof=1):.2f}")
print(f"Fertilizer 2: mean={np.mean(fertilizer2):.2f}, std={np.std(fertilizer2, ddof=1):.2f}")
print(f"Fertilizer 3: mean={np.mean(fertilizer3):.2f}, std={np.std(fertilizer3, ddof=1):.2f}")
print(f"Fertilizer 4: mean={np.mean(fertilizer4):.2f}, std={np.std(fertilizer4, ddof=1):.2f}")

# One-way ANOVA untuk agricultural research
f_stat_ag, p_value_ag = f_oneway(fertilizer1, fertilizer2, fertilizer3, fertilizer4)
print(f"F-statistic: {f_stat_ag:.4f}")
print(f"p-value: {p_value_ag:.4f}")

# 6. Visualisasi Aplikasi Praktis ANOVA
plt.figure(figsize=(20, 15))

# Plot 1: A/B Testing
plt.subplot(3, 4, 1)
ab_data = [website_a, website_b, website_c]
ab_labels = ['Website A', 'Website B', 'Website C']
plt.boxplot(ab_data, labels=ab_labels)
plt.ylabel('Conversion Rate')
plt.title('A/B Testing - Website Conversion')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 2: Quality Control
plt.subplot(3, 4, 2)
qc_data = [machine1, machine2, machine3, machine4]
qc_labels = ['Machine 1', 'Machine 2', 'Machine 3', 'Machine 4']
plt.boxplot(qc_data, labels=qc_labels)
plt.ylabel('Quality Score')
plt.title('Quality Control - Machine Performance')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 3: Clinical Trial
plt.subplot(3, 4, 3)
ct_data = [drug_a, drug_b, drug_c]
ct_labels = ['Drug A', 'Drug B', 'Drug C']
plt.boxplot(ct_data, labels=ct_labels)
plt.ylabel('Recovery Time (days)')
plt.title('Clinical Trial - Drug Effectiveness')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 4: Educational Research
plt.subplot(3, 4, 4)
ed_data = [method1, method2, method3]
ed_labels = ['Method 1', 'Method 2', 'Method 3']
plt.boxplot(ed_data, labels=ed_labels)
plt.ylabel('Student Score')
plt.title('Educational Research - Teaching Methods')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 5: Agricultural Research
plt.subplot(3, 4, 5)
ag_data = [fertilizer1, fertilizer2, fertilizer3, fertilizer4]
ag_labels = ['Fertilizer 1', 'Fertilizer 2', 'Fertilizer 3', 'Fertilizer 4']
plt.boxplot(ag_data, labels=ag_labels)
plt.ylabel('Crop Yield (kg)')
plt.title('Agricultural Research - Fertilizer Effectiveness')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 6: Mean comparison untuk semua aplikasi
plt.subplot(3, 4, 6)
applications = ['A/B Testing', 'Quality Control', 'Clinical Trial', 'Educational', 'Agricultural']
f_stats = [f_stat_ab, f_stat_qc, f_stat_ct, f_stat_ed, f_stat_ag]
plt.bar(applications, f_stats, alpha=0.7, color=['blue', 'green', 'red', 'orange', 'purple'])
plt.ylabel('F-statistic')
plt.title('F-statistics untuk Berbagai Aplikasi')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 7: P-value comparison
plt.subplot(3, 4, 7)
p_values = [p_value_ab, p_value_qc, p_value_ct, p_value_ed, p_value_ag]
plt.bar(applications, p_values, alpha=0.7, color=['blue', 'green', 'red', 'orange', 'purple'])
plt.axhline(0.05, color='red', linestyle='--', label='α = 0.05')
plt.ylabel('P-value')
plt.title('P-values untuk Berbagai Aplikasi')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 8: Effect sizes
plt.subplot(3, 4, 8)
# Hitung effect size untuk setiap aplikasi
def calculate_eta_squared(groups):
    all_data = np.concatenate(groups)
    grand_mean = np.mean(all_data)
    group_means = [np.mean(group) for group in groups]
    group_sizes = [len(group) for group in groups]
    
    ss_between = sum([n * (mean - grand_mean)**2 for n, mean in zip(group_sizes, group_means)])
    ss_total = sum((all_data - grand_mean)**2)
    
    return ss_between / ss_total

eta_squared_values = [
    calculate_eta_squared([website_a, website_b, website_c]),
    calculate_eta_squared([machine1, machine2, machine3, machine4]),
    calculate_eta_squared([drug_a, drug_b, drug_c]),
    calculate_eta_squared([method1, method2, method3]),
    calculate_eta_squared([fertilizer1, fertilizer2, fertilizer3, fertilizer4])
]

plt.bar(applications, eta_squared_values, alpha=0.7, color=['blue', 'green', 'red', 'orange', 'purple'])
plt.ylabel('Eta-squared (η²)')
plt.title('Effect Sizes untuk Berbagai Aplikasi')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 9: Sample sizes
plt.subplot(3, 4, 9)
sample_sizes = [len(website_a), len(machine1), len(drug_a), len(method1), len(fertilizer1)]
plt.bar(applications, sample_sizes, alpha=0.7, color=['blue', 'green', 'red', 'orange', 'purple'])
plt.ylabel('Sample Size')
plt.title('Sample Sizes untuk Berbagai Aplikasi')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# Plot 10: Power analysis
plt.subplot(3, 4, 10)
# Simulasi power analysis
effect_sizes = np.linspace(0.1, 1.0, 100)
powers = []
for es in effect_sizes:
    # Simulasi power calculation
    power = 1 - f.cdf(f.ppf(0.95, 2, 100), 2, 100)  # df_between=2, df_within=100
    powers.append(power)

plt.plot(effect_sizes, powers, 'b-', linewidth=2)
plt.axhline(0.8, color='red', linestyle='--', label='Power = 0.8')
plt.xlabel('Effect Size')
plt.ylabel('Power')
plt.title('Power Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 11: Summary statistics
plt.subplot(3, 4, 11)
plt.axis('off')
summary_text = f"""
APLIKASI PRAKTIS ANOVA SUMMARY

A/B Testing:
F = {f_stat_ab:.4f}, p = {p_value_ab:.4f}
η² = {eta_squared_values[0]:.4f}

Quality Control:
F = {f_stat_qc:.4f}, p = {p_value_qc:.4f}
η² = {eta_squared_values[1]:.4f}

Clinical Trial:
F = {f_stat_ct:.4f}, p = {p_value_ct:.4f}
η² = {eta_squared_values[2]:.4f}

Educational:
F = {f_stat_ed:.4f}, p = {p_value_ed:.4f}
η² = {eta_squared_values[3]:.4f}

Agricultural:
F = {f_stat_ag:.4f}, p = {p_value_ag:.4f}
η² = {eta_squared_values[4]:.4f}
"""
plt.text(0.1, 0.9, summary_text, transform=plt.gca().transAxes, fontsize=8, 
         verticalalignment='top', fontfamily='monospace')

# Plot 12: Recommendations
plt.subplot(3, 4, 12)
plt.axis('off')
recommendations_text = f"""
REKOMENDASI APLIKASI

1. A/B Testing:
   - Gunakan untuk membandingkan versi
   - Perhatikan sample size
   - Pertimbangkan practical significance

2. Quality Control:
   - Monitor proses produksi
   - Identifikasi mesin bermasalah
   - Implementasi perbaikan

3. Clinical Trial:
   - Pastikan ethical approval
   - Gunakan random assignment
   - Pertimbangkan placebo effect

4. Educational Research:
   - Pertimbangkan confounding variables
   - Gunakan control group
   - Ukur long-term effects

5. Agricultural Research:
   - Pertimbangkan kondisi lingkungan
   - Gunakan replication
   - Ukur multiple outcomes
"""
plt.text(0.1, 0.9, recommendations_text, transform=plt.gca().transAxes, fontsize=8, 
         verticalalignment='top', fontfamily='monospace')

plt.tight_layout()
plt.show()

# 7. Interpretasi Hasil
print("\n7. INTERPRETASI HASIL:")

print(f"\nA/B Testing:")
if p_value_ab < 0.05:
    print(f"  - Signifikan (p = {p_value_ab:.4f} < 0.05)")
    print("  - Ada perbedaan conversion rate antar website")
    print("  - Rekomendasi: Pilih website dengan conversion rate tertinggi")
else:
    print(f"  - Tidak signifikan (p = {p_value_ab:.4f} >= 0.05)")
    print("  - Tidak ada perbedaan conversion rate antar website")

print(f"\nQuality Control:")
if p_value_qc < 0.05:
    print(f"  - Signifikan (p = {p_value_qc:.4f} < 0.05)")
    print("  - Ada perbedaan kualitas produk antar mesin")
    print("  - Rekomendasi: Investigasi mesin dengan kualitas rendah")
else:
    print(f"  - Tidak signifikan (p = {p_value_qc:.4f} >= 0.05)")
    print("  - Tidak ada perbedaan kualitas produk antar mesin")

print(f"\nClinical Trial:")
if p_value_ct < 0.05:
    print(f"  - Signifikan (p = {p_value_ct:.4f} < 0.05)")
    print("  - Ada perbedaan recovery time antar obat")
    print("  - Rekomendasi: Pilih obat dengan recovery time terpendek")
else:
    print(f"  - Tidak signifikan (p = {p_value_ct:.4f} >= 0.05)")
    print("  - Tidak ada perbedaan recovery time antar obat")

print(f"\nEducational Research:")
if p_value_ed < 0.05:
    print(f"  - Signifikan (p = {p_value_ed:.4f} < 0.05)")
    print("  - Ada perbedaan nilai siswa antar metode pembelajaran")
    print("  - Rekomendasi: Implementasi metode dengan nilai tertinggi")
else:
    print(f"  - Tidak signifikan (p = {p_value_ed:.4f} >= 0.05)")
    print("  - Tidak ada perbedaan nilai siswa antar metode pembelajaran")

print(f"\nAgricultural Research:")
if p_value_ag < 0.05:
    print(f"  - Signifikan (p = {p_value_ag:.4f} < 0.05)")
    print("  - Ada perbedaan hasil panen antar pupuk")
    print("  - Rekomendasi: Gunakan pupuk dengan hasil panen tertinggi")
else:
    print(f"  - Tidak signifikan (p = {p_value_ag:.4f} >= 0.05)")
    print("  - Tidak ada perbedaan hasil panen antar pupuk")

# 8. Kesimpulan dan Rekomendasi
print("\n8. KESIMPULAN DAN REKOMENDASI:")
print("   - ANOVA: Aplikasi luas dalam berbagai bidang penelitian")
print("   - A/B Testing: Efektif untuk membandingkan versi produk")
print("   - Quality Control: Penting untuk monitoring proses produksi")
print("   - Clinical Trial: Kritis untuk pengembangan obat")
print("   - Educational Research: Membantu peningkatan metode pembelajaran")
print("   - Agricultural Research: Meningkatkan produktivitas pertanian")
print("   - Best Practices: Perhatikan asumsi, sample size, dan effect size")
print("   - Interpretasi: Pertimbangkan practical significance, bukan hanya statistical significance")


## 8. Best Practices dan Troubleshooting

### 8.1 Best Practices untuk ANOVA

#### 8.1.1 Perencanaan Penelitian
- **Tentukan hipotesis yang jelas**: Sebelum mengumpulkan data
- **Hitung sample size**: Gunakan power analysis untuk menentukan n yang memadai
- **Pilih desain yang tepat**: One-way vs Two-way ANOVA
- **Definisikan variabel**: Pastikan variabel dependen dan independen jelas

#### 8.1.2 Pengumpulan Data
- **Random sampling**: Pastikan sampel representatif
- **Kontrol confounding variables**: Identifikasi dan kontrol variabel yang mempengaruhi
- **Konsistensi pengukuran**: Gunakan instrumen yang sama untuk semua kelompok
- **Dokumentasi**: Catat semua detail pengumpulan data

#### 8.1.3 Analisis Data
- **Periksa asumsi**: Normalitas, homogenitas varian, independensi
- **Gunakan software yang tepat**: R, Python, SPSS, atau software statistik lainnya
- **Hitung effect size**: Jangan hanya mengandalkan p-value
- **Lakukan post-hoc analysis**: Jika ANOVA signifikan

#### 8.1.4 Interpretasi Hasil
- **Pertimbangkan practical significance**: Bukan hanya statistical significance
- **Gunakan confidence intervals**: Untuk estimasi parameter
- **Laporkan effect size**: Eta-squared, Cohen's d, dll.
- **Pertimbangkan konteks**: Interpretasi dalam konteks penelitian

### 8.2 Troubleshooting ANOVA

#### 8.2.1 Pelanggaran Asumsi Normalitas
- **Gejala**: Data tidak berdistribusi normal
- **Penyebab**: Skewness, kurtosis, outliers
- **Solusi**: 
  - Transformasi data (log, square root, Box-Cox)
  - Uji non-parametrik (Kruskal-Wallis)
  - Robust ANOVA

#### 8.2.2 Pelanggaran Asumsi Homogenitas Varian
- **Gejala**: Varian antar kelompok berbeda signifikan
- **Penyebab**: Heteroscedasticity
- **Solusi**:
  - Welch's ANOVA
  - Uji non-parametrik (Kruskal-Wallis)
  - Transformasi data

#### 8.2.3 Pelanggaran Asumsi Independensi
- **Gejala**: Observasi tidak independen
- **Penyebab**: Clustering, repeated measures
- **Solusi**:
  - Mixed-effects models
  - Repeated measures ANOVA
  - Generalized linear models

#### 8.2.4 Outliers
- **Gejala**: Data point yang ekstrem
- **Penyebab**: Measurement error, data entry error
- **Solusi**:
  - Investigasi outlier
  - Transformasi data
  - Robust statistics
  - Exclude outlier (dengan justifikasi)

#### 8.2.5 Missing Data
- **Gejala**: Data tidak lengkap
- **Penyebab**: Non-response, equipment failure
- **Solusi**:
  - Complete case analysis
  - Multiple imputation
  - Maximum likelihood estimation

### 8.3 Common Mistakes

#### 8.3.1 Kesalahan dalam Perencanaan
- **Tidak menghitung sample size**: Menyebabkan power rendah
- **Tidak menentukan hipotesis**: Menyebabkan fishing expedition
- **Tidak mempertimbangkan confounding**: Menyebabkan bias

#### 8.3.2 Kesalahan dalam Analisis
- **Tidak memeriksa asumsi**: Menyebabkan hasil tidak valid
- **Menggunakan t-test berulang**: Menyebabkan inflasi Type I error
- **Tidak melakukan post-hoc**: Tidak mengetahui kelompok mana yang berbeda

#### 8.3.3 Kesalahan dalam Interpretasi
- **Hanya melihat p-value**: Mengabaikan effect size
- **Mengabaikan practical significance**: Fokus pada statistical significance
- **Over-interpretasi hasil**: Menyimpulkan lebih dari yang seharusnya

### 8.4 Software dan Tools

#### 8.4.1 R
- **Paket**: `aov()`, `lm()`, `car`, `emmeans`
- **Keunggulan**: Gratis, powerful, banyak paket
- **Kekurangan**: Learning curve tinggi

#### 8.4.2 Python
- **Paket**: `scipy.stats`, `statsmodels`, `pingouin`
- **Keunggulan**: Mudah dipelajari, integrasi dengan data science
- **Kekurangan**: Lebih sedikit paket statistik

#### 8.4.3 SPSS
- **Fitur**: GUI, output lengkap, mudah digunakan
- **Keunggulan**: User-friendly, output profesional
- **Kekurangan**: Berbayar, kurang fleksibel

#### 8.4.4 SAS
- **Fitur**: Powerful, enterprise-grade
- **Keunggulan**: Sangat powerful, banyak fitur
- **Kekurangan**: Mahal, learning curve tinggi

### 8.5 Reporting Guidelines

#### 8.5.1 Informasi yang Harus Dilaporkan
- **Desain penelitian**: One-way atau Two-way ANOVA
- **Sample size**: Jumlah observasi per kelompok
- **Asumsi**: Hasil pengecekan asumsi
- **Statistik**: F-statistic, degrees of freedom, p-value
- **Effect size**: Eta-squared, Cohen's d, dll.
- **Post-hoc**: Hasil analisis post-hoc jika signifikan

#### 8.5.2 Format Pelaporan
- **Teks**: "One-way ANOVA menunjukkan perbedaan signifikan antar kelompok, F(3, 116) = 15.67, p < 0.001, η² = 0.29"
- **Tabel**: ANOVA table dengan semua informasi
- **Grafik**: Box plot, bar chart, atau plot lainnya

#### 8.5.3 Interpretasi yang Benar
- **Statistical significance**: p < α
- **Practical significance**: Effect size yang bermakna
- **Confidence intervals**: Estimasi parameter
- **Limitations**: Keterbatasan penelitian

### 8.6 Advanced Topics

#### 8.6.1 Mixed-Effects ANOVA
- **Tujuan**: Mengatasi pelanggaran independensi
- **Aplikasi**: Repeated measures, nested designs
- **Software**: R (lme4), Python (statsmodels)

#### 8.6.2 Robust ANOVA
- **Tujuan**: Mengatasi outliers dan non-normality
- **Metode**: Trimmed means, M-estimators
- **Software**: R (WRS2), Python (scipy.stats)

#### 8.6.3 Bayesian ANOVA
- **Tujuan**: Probabilistic inference
- **Keunggulan**: Interpretasi yang lebih intuitif
- **Software**: R (rstanarm), Python (PyMC3)

#### 8.6.4 Power Analysis
- **Tujuan**: Menentukan sample size yang memadai
- **Metode**: A priori, post hoc, sensitivity
- **Software**: G*Power, R (pwr), Python (statsmodels)

### 8.7 Checklist ANOVA

#### 8.7.1 Sebelum Analisis
- [ ] Hipotesis sudah jelas
- [ ] Sample size memadai
- [ ] Desain penelitian tepat
- [ ] Data sudah dikumpulkan dengan benar

#### 8.7.2 Selama Analisis
- [ ] Periksa asumsi normalitas
- [ ] Periksa asumsi homogenitas varian
- [ ] Periksa asumsi independensi
- [ ] Identifikasi outliers
- [ ] Lakukan ANOVA
- [ ] Hitung effect size
- [ ] Lakukan post-hoc jika signifikan

#### 8.7.3 Setelah Analisis
- [ ] Interpretasi hasil dengan benar
- [ ] Laporkan semua statistik yang relevan
- [ ] Pertimbangkan practical significance
- [ ] Diskusikan keterbatasan
- [ ] Berikan rekomendasi

### 8.8 Resources dan Referensi

#### 8.8.1 Buku Teks
- "Design and Analysis of Experiments" - Montgomery
- "Statistical Methods for Psychology" - Howell
- "Applied Statistics for the Behavioral Sciences" - Hinkle

#### 8.8.2 Online Resources
- G*Power (Power analysis)
- R Documentation
- Python SciPy Documentation
- Statistics How To

#### 8.8.3 Courses
- Coursera: Statistics courses
- edX: Data Science courses
- Udemy: Statistics courses
- YouTube: Statistics tutorials
